# Library

In [ ]:
# @title Imports
import numpy as np
import cupy as cp
from scipy import signal

from tqdm.auto import tqdm
import time
import os

import h5py as h5
import pandas as pd

import soundfile as sf
import urllib.request
        
import cv2

from scipy.signal import butter, lfilter, filtfilt

import matplotlib.pyplot as plt
import matplotlib.animation as ani
import matplotlib.gridspec as gridspec
from matplotlib.patches import Circle

import imageio
from IPython.display import HTML
from base64 import b64encode

%matplotlib widget

In [ ]:
def butter_highpass_filter(data, cut, fs, order=4, repeat=4):
    def butter_highpass(cut, fs, order=4):
        nyq = 0.5 * fs
        cut = cut / nyq
        b, a = butter(order, cut, btype='highpass')
        return b, a

    b, a = butter_highpass(cut, fs, order=order)
    y = data
    for _ in range(repeat):
        y = filtfilt(b, a, y)
    return y
def load_from_h5(filename, verbose=0):
    """
    The function loads a data structure from a hdf5 file to list or dictionary
    """

    def recursively_load_contents_from_group(h5file, path):
        """
        load dictionaries or iterables
        """

        key = list(h5file[path].keys())[0]
        isnumber = False
        try:
            int(key)  # this fails for non-integer strings
            isnumber = True
        except ValueError:
            pass

        # iterables
        if isnumber:
            ans = list()
            keys = [key for key in h5file[path].keys()]
            keys_argsorted = np.argsort([int(key) for key in keys])
            for keyx in keys_argsorted:
                key = keys[keyx]
                item = h5file[path][key]
                if isinstance(item, h5._hl.dataset.Dataset):
                    if verbose > 0:
                        print('loading: {}/{}'.format(path, key))
                    ans.append(item[()])
                elif isinstance(item, h5._hl.group.Group):
                    ans.append(recursively_load_contents_from_group(h5file, path + key + '/'))
        # dictionaries
        else:
            ans = dict()
            for key, item in h5file[path].items():
                if 'pandas_type' in item.attrs.keys():
                    ans[key] = pd.read_hdf(filename, key)
                elif isinstance(item, h5._hl.dataset.Dataset):
                    if verbose > 0:
                        print('loading: {}/{}'.format(path, key))
                    ans[key] = item[()]
                elif isinstance(item, h5._hl.group.Group):
                    ans[key] = recursively_load_contents_from_group(h5file, path + key + '/')
        return ans

    with h5.File(filename, 'r') as h5file:
        return recursively_load_contents_from_group(h5file, '/')
    

In [ ]:
# @title Sound Library

### SOUNDS ####
def np_gaussian_double_pulse(samplerate, center_frequency=5000, pulse_duration=0.001):
    """
    Creates a gausian double pulse with a given center frequency
    """
    tt = np.arange(0, int(pulse_duration * samplerate), dtype=float)
    wav = np.sin(1 * cp.pi * 1e3 * tt / samplerate)
    ts = 1 / (2 * np.pi * center_frequency) * samplerate
    ct = (pulse_duration * samplerate) // 2  # 10 * ts
    wav = (tt - ct) / ts * np.exp(-((tt - ct) / ts) ** 2 / 2)
    pulse = wav / wav.max()
    return pulse

def np_deltapulse(samplerate, duration=0.01, offset=0.001):
    """
    Create a kronecker delta pulse
    """
    nsp = int(duration * samplerate)
    start = int(offset * samplerate)
    wav = np.zeros(nsp)
    wav[start] = 1
    return wav

def np_ramped_sinusoid(samplerate, freq=200, duration=0.1, ramp_time=0.01):
    """
    Creates a hann-ramped steady sinusoid
    """
    ramp_samples = int(ramp_time * samplerate)
    tt = np.arange(0, int(duration * samplerate), dtype=float)
    wav = np.sin(2 * np.pi * freq * tt / samplerate)
    hann = signal.windows.hann(2 * ramp_samples + 1)
    wav[:ramp_samples] *= hann[:ramp_samples]
    wav[-ramp_samples:] *= hann[-ramp_samples:]
    return wav / wav.max()

In [ ]:
# @title Mesh Definitions

### MESH DEFINITIONS ###
def create_mesh1(dx=2.5e-3,Lx=100e-2,Ly=100e-2,Lz=50e-2,sr_target=1e6,water_height=10e-2,wall_height=20e-2,wall_d=5e-3,glass_d=1.5e-2,inner_sz=30e-2,outer_sz=35e-2):
  """
  Rectangular pmma tank with water on glass in air
  """
  # configure simulation parameters (all in SI units)
  xx=np.r_[-Lx/2:Lx/2:dx].astype('single')+dx/2
  yy=np.r_[-Ly/2:Ly/2:dx].astype('single')+dx/2
  zz=np.r_[-Lz/2:Lz/2:dx].astype('single')+dx/2
  X,Y,Z=np.meshgrid(xx,yy,zz)

  wtrh=water_height
  volume_shape = np.array(X.shape)

  # air
  c = np.ones(volume_shape) * 343
  rho = np.ones(volume_shape) * 1.2
  # water
  mask_water = np.zeros(volume_shape, dtype='bool')
  mask_water = np.logical_and(np.maximum(np.abs(X),np.abs(Y))<=inner_sz,np.abs(Z)<=(wtrh/2))
  c[mask_water] = 1480
  rho[mask_water] = 1000
  #glass 
  mask_glass =  np.logical_and(np.maximum(np.abs(X),np.abs(Y))<outer_sz+1e-2,(Z<(-wtrh/2-wall_d))*(Z>=(-wtrh/2-glass_d-wall_d)))
  c[mask_glass] = 4540
  rho[mask_glass] = 2500
  #PMMA
  mask_pmma =  (np.maximum(np.abs(X),np.abs(Y))<inner_sz+wall_d) *  (Z<(-wtrh/2)) *  (Z>=(-wtrh/2-wall_d))
  mask_pmma+= (np.maximum(np.abs(X),np.abs(Y))>=inner_sz) * (np.maximum(np.abs(X),np.abs(Y))<inner_sz+wall_d) *(np.abs(Z-5e-2)<=(wall_height/2))

  c[mask_pmma] = 2700
  rho[mask_pmma] = 1200

  mesh = {'volume_shape': volume_shape,
          'dx': dx,
          'c': c,
          'rho': rho}
                
  return mesh

def create_mesh2():
  """
  Cylindrical water in air
  """
  # box
  dx = 0.01
  l0 = 33
  l1 = 33
  l2 = 25
  volume_shape = np.array([l0, l1, l2])

  # air
  c = np.ones(volume_shape) * 343
  rho = np.ones(volume_shape) * 1.2

  # water
  wr = 12  # radius
  wh = 5  # height
  g0, g1 = np.ogrid[:l0, :l1]
  dis = np.sqrt((g0 - l0 // 2) ** 2 + (g1 - l1 // 2) ** 2)
  circle = dis <= wr
  startz = (l2 - wh) // 2

  # physics
  c_h2o = 1480
  rho_h2o = 1000
  c[circle, startz:startz + wh] = c_h2o
  rho[circle,] = rho_h2o

  # positions of micro/hydrophones and speakers
  p_emitters = ((12, 16, 12), (20, 16, 12))
  p_sensors = ((14, 16, 12), (16, 16, 12), (18, 16, 12))

  mesh = {'volume_shape': volume_shape,
                'dx': dx,
                'c': c,
                'rho': rho,
                'p_emitters': p_emitters,
                'p_sensors': p_sensors}

  return mesh

def create_mesh3(dx=5e-3,Lx=49e-2,Ly=49e-2,Lz=31e-2,water_height=12e-2, water_width=30e-2, with_sensors = False, with_emitters=False):
  """
  Rectangular block of water in air with optional sensor array
  """

  # configure simulation parameters (all in SI units)
  xx=np.r_[-Lx/2:Lx/2:dx].astype('single')+dx/2
  yy=np.r_[-Ly/2:Ly/2:dx].astype('single')+dx/2
  zz=np.r_[-Lz/2:Lz/2:dx].astype('single')+dx/2
  X,Y,Z=np.meshgrid(xx,yy,zz)

  wtrh=water_height
  volume_shape = np.array(X.shape)

  # air
  c = np.ones(volume_shape) * 343
  rho = np.ones(volume_shape) * 1.2
  # water
  mask_water = np.zeros(volume_shape, dtype='bool')
  mask_water = np.logical_and(np.maximum(np.abs(X),np.abs(Y))<=water_width/2,np.abs(Z)<=(wtrh/2))
  c[mask_water] = 1480
  rho[mask_water] = 1000

  mesh = {'volume_shape': volume_shape,
          'dx': dx,
          'c': c,
          'rho': rho}   

  cx, cy, cz = np.array(volume_shape)//2
  # positions of micro/hydrophones 
  if with_sensors:
    sens_distance_from_center = 3e-2
    sdfc = int(sens_distance_from_center/dx)
    p_sensors =  ((cx,cy,cz),(cx-sdfc,cy,cz),(cx+sdfc,cy,cz),(cx,cy-sdfc,cz),(cx,cy+sdfc,cz))
    mesh['p_sensors'] = p_sensors

  # positions of speakers
  if with_emitters:
    emit_distance_from_center = 8e-2
    edfc = int(emit_distance_from_center/dx)
    p_emitters = ((cx-edfc,cy,cz),(cx+edfc,cy,cz),(cx,cy-edfc,cz),(cx,cy+edfc,cz))
    mesh['p_emitters'] = p_emitters

  return mesh

In [ ]:
# @title WeeFDTD Class

### SIMULATION ###
class WeeFDTD:
  def __init__(self, mesh):
    """
    definition of mesh
    mesh = {'volume_shape': volume_shape,
                  'dx': dx,
                  'c': c,
                  'rho': rho,
                  'p_emitters': p_emitters,
                  'p_sensors': p_sensors}

    p_sensors is optional.
    """
    # simulation sampling rate
    self.mesh = mesh
    self.dt = float(mesh['dx'] / mesh['c'].max() / 3)  # Courant condition
    self.sr = 1 / self.dt
    print(f"Simulation samplerate: {self.sr}Hz")

  def wee_fdtd(self, dur, record_fcn, stim_fcn, pml_size=10, pml_gain=1 / 10):
    """GPU-accelerated 3D FDTD acoustic simulator with PML boundary.
    Args:
        volume_shape: shape of the simulation volume
        dx: isotropic voxel size (m)
        rho: 3D density map (kg/m^3)
        c: 3D speed map (m/s)
        dur: simulation duration (s)
        record_fcn: recording callback with args (p, u_list, iStep), returning variables to be recorded
        stim_fcn: stimulation callback with args (p_list, iStep, dt), operating on a list of pressure map components
        dt: time step (defaults to Courant condition)
        pml_size: size of PML border in voxels (defaults to 10)
        pml_gain: tune this to optimize absorption (defaults to 1/10)
    Returns:
        output: list of outputs generated by record_fcn
        dt: time step"""

    dx = self.mesh['dx']
    volume_shape = self.mesh['volume_shape']
    rho = cp.asarray(self.mesh['rho'])   # to GPU
    c = cp.asarray(self.mesh['c'])  # to GPU
    dt = self.dt
    assert dt <= float(dx / c.max() / 3)  # choose a dt smaller than Courant condition

    p = p0 = p1 = p2 = cp.zeros(volume_shape)  # pressure map
    u0 = cp.zeros(volume_shape - [1, 0, 0])  # velocity field components
    u1 = cp.zeros(volume_shape - [0, 1, 0])
    u2 = cp.zeros(volume_shape - [0, 0, 1])
    coU = (dt / dx) * rho * c * c  # pressure coefficient
    coP = (dt / dx) / rho  # velocity coefficient
    pml_ramp = cp.arange(1, pml_size + 1) / pml_size
    q0 = 1 - pml_gain * cp.r_[cp.flip(pml_ramp, 0), cp.zeros(int(volume_shape[0] - 2 * pml_size)), pml_ramp][:,
                        None, None]
    q1 = 1 - pml_gain * cp.r_[cp.flip(pml_ramp, 0), cp.zeros(int(volume_shape[1] - 2 * pml_size)), pml_ramp][None,
                        :, None]
    q2 = 1 - pml_gain * cp.r_[cp.flip(pml_ramp, 0), cp.zeros(int(volume_shape[2] - 2 * pml_size)), pml_ramp][None,
                        None, :]
    q0u = (q0[1:, :, :] + q0[:-1, :, :]) / 2
    q1u = (q1[:, 1:, :] + q1[:, :-1, :]) / 2
    q2u = (q2[:, :, 1:] + q2[:, :, :-1]) / 2

    # GPU memory management
    mempool = cp.get_default_memory_pool()
    pinned_pool = cp.get_default_pinned_memory_pool()
    limit = 2 * 1024**3 #in GiB
    print(f"Limits GPU memory to approx. {limit} Bytes.")
    output_gpu = []

    first = True
    for iStep in tqdm(range(int(dur / dt))):
        coPxP = coP * p
        u0 = u0 * q0u - cp.diff(coPxP, axis=0)
        u1 = u1 * q1u - cp.diff(coPxP, axis=1)
        u2 = u2 * q2u - cp.diff(coPxP, axis=2)
        p0 = p0 * q0 - coU * cp.diff(u0, axis=0, prepend=0, append=0)
        p1 = p1 * q1 - coU * cp.diff(u1, axis=1, prepend=0, append=0)
        p2 = p2 * q2 - coU * cp.diff(u2, axis=2, prepend=0, append=0)
        stim_fcn([p0, p1, p2], iStep, dt)
        p = p0 + p1 + p2
        output_gpu.append(record_fcn(p, [u0, u1, u2], iStep).copy())
        # todo: check how costly a call to used_bytes is. Check it at each step?
        if mempool.used_bytes() > limit:
            print(f"Reset GPU memory after using {mempool.used_bytes()} Bytes")
            if first:
                output_cpu = cp.array(output_gpu).get()
                first = False
            else:
                output_cpu = np.vstack((output_cpu, cp.array(output_gpu).get()))
            output_gpu = []
            #mempool.free_all_blocks()
            #pinned_pool.free_all_blocks()
            time.sleep(.01) # wait for freeing
    if first:
        output_cpu = cp.array(output_gpu).get()
    else:
        output_cpu = np.vstack((output_cpu, cp.array(output_gpu).get()))
    return output_cpu

  def rec_p_sensors(self, sd, sd_sr):
    assert self.mesh['p_sensors'] # need to define pressure sensor positions
    assert self.mesh['p_emitters'] # need to define pressure emitter positions

    p_sensors = self.mesh['p_sensors']
    p_emitters =  self.mesh['p_emitters']
    resample = self.sr != sd_sr

    if resample:
      rsmpl_ratio = self.sr / sd_sr
      orig_length = sd.shape[0]
      sd = signal.resample(sd, num=int(orig_length * rsmpl_ratio), axis=0)
      print(f"Simulation samplerate: {self.sr}Hz")
      print(f"Data samplerate: {sd_sr}Hz")
      print("Resampled sound to match simulation samplerate")

    # pressure sensors
    if cp.ndim(p_sensors)>1: #multiple
      record_fcn = lambda p, u_list, iStep: [p[idx] for idx in p_sensors]
    else: #single
      record_fcn = lambda p, u_list, iStep: [p[p_sensors]]

    # pressure emitters
    sd = cp.array(sd)
    dur_stim = sd.shape[0] * self.dt

    if cp.ndim(p_emitters)>1:
      def stim_fcn(p_list, iStep, dt):
          for p_i in p_list:
              for m in range(len(p_emitters)):
                  p_i[p_emitters[m]] = sd[iStep,m] / 3
    else:
      def stim_fcn(p_list, iStep, dt):
          for p_i in p_list:
                p_i[p_emitters] = sd[iStep] / 3

    # run
    p_rec = self.wee_fdtd(dur_stim, record_fcn=record_fcn, stim_fcn=stim_fcn)

    if resample:
      p_rec = signal.resample(p_rec, num=orig_length, axis=0)
      print("Resampled simulation result back to sound samplerate.")
    return p_rec

  def rec_pv_plane(self, sd, sd_sr, plane_axis=None, plane_pos=None):
    if plane_axis is None or plane_pos is None:
      print("Record z slice at half height of simulation chamber.")
      plane_axis = 2
      plane_pos = self.mesh['volume_shape'][2]//2

    assert self.mesh['p_emitters'] # need to define pressure emitter positions
    p_emitters =  self.mesh['p_emitters']
    resample = self.sr != sd_sr

    if resample:
      rsmpl_ratio = self.sr / sd_sr
      orig_length = sd.shape[0]
      sd = signal.resample(sd, num=int(orig_length * rsmpl_ratio), axis=0)
      print(f"Simulation samplerate: {self.sr}Hz")
      print(f"Data samplerate: {sd_sr}Hz")
      print("Resampled sound to match simulation samplerate")

    # sensors
    if plane_axis==0:
      record_fcn = lambda p, u_list, iStep: cp.concatenate((p[plane_pos,:-1,:-1],u_list[1][plane_pos,:,:-1],u_list[2][plane_pos,:-1,:]))
    elif plane_axis==1:
      record_fcn = lambda p, u_list, iStep: cp.concatenate((p[:-1,plane_pos,:-1],u_list[0][:,plane_pos,:-1],u_list[2][:-1,plane_pos,:]))
    elif plane_axis==2:
      record_fcn = lambda p, u_list, iStep: cp.concatenate((p[:-1,:-1,plane_pos],u_list[0][:,:-1,plane_pos],u_list[1][:-1,:,plane_pos]))
    else:
      print("Choose plane axis 0, 1 or 2.")

    # pressure emitters
    sd = cp.array(sd)
    dur_stim = sd.shape[0] * self.dt

    if cp.ndim(p_emitters)>1:
      def stim_fcn(p_list, iStep, dt):
          for p_i in p_list:
              for m in range(len(p_emitters)):
                  p_i[p_emitters[m]] = sd[iStep,m] / 3
    else:
      def stim_fcn(p_list, iStep, dt):
          for p_i in p_list:
                p_i[p_emitters] = sd[iStep] / 3

    # run
    data = self.wee_fdtd(dur_stim, record_fcn=record_fcn, stim_fcn=stim_fcn)

    if resample:
      data = signal.resample(data, num=orig_length, axis=0)
      print("Resampled simulation result back to sound samplerate.")

    # unstack measurements
    # data: time x (dim0 p + dim0 v)  x dim1
    if plane_axis==0:
      dim0 = self.mesh['volume_shape'][1] - 1
    else:
      dim0 = self.mesh['volume_shape'][0] - 1

    p = data[:,:dim0,:]
    v0 = data[:,dim0:2*dim0,:]
    v1 = data[:,2*dim0:,:]
    return p, v0, v1


In [ ]:
# @title Sound Conditioning

def record_irs_p_sensors(Sim, sd_sr, duration=.01, offset=.001):
  """
  Sim: simulation class
  sd_sr: samplerate for deltapulse
  duration: length of sound (s)
  offset: deltapulse located at offset (s)
  """
  irs_meta = locals()
  del irs_meta['Sim']

  dp = np_deltapulse(sd_sr, duration, offset)

  irs = []
  nes = len(Sim.mesh['p_emitters'])
  for i in range(nes):
    sd = np.zeros((dp.size, nes))  # n_speakers X length_deltapulse
    sd[:, i] = dp
    irs.append(Sim.rec_p_sensors(sd, sd_sr))
  return np.array(irs), irs_meta


def record_irs_pv_plane(Sim, sd_sr, duration=.01, offset=.001, plane_axis=1, plane_pos=None):
  """
  Sim: simulation class
  sd_sr: samplerate for deltapulse
  duration: length of sound (s)
  offset: deltapulse located at offset (s)
  """
  irs_meta = locals()
  del irs_meta['Sim']

  dp = np_deltapulse(sd_sr, duration, offset)

  irs_p = []
  irs_v0 = []
  irs_v1 = []
  nes = len(Sim.mesh['p_emitters'])
  for i in range(nes):
    sd = np.zeros((dp.size, nes))  # n_speakers X length_deltapulse
    sd[:, i] = dp
    p, v0, v1 = Sim.rec_pv_plane(sd, sd_sr,plane_axis,plane_pos)
    irs_p.append(p)
    irs_v0.append(v0)
    irs_v1.append(v1)
  return np.array(irs_p), np.array(irs_v0), np.array(irs_v1), irs_meta

def make_kernels(irs, irs_meta, obs):
  # turn recording into observables
  kernels = {}
  offset = int(irs_meta['offset']*irs_meta['sd_sr'])
  for i, (key, func) in enumerate(sorted(obs.items())):
      ir_obs = [func(ir[offset:]) for ir in irs]
      kernels[key] = ir_obs
  return kernels

def find_sound(ks, tg, sp):
    """
    Computes sound that can generate the desired target signals, given kernels.
    Kernels are given for each speaker. However, one could couple several speakers, a mapping which
    is defined by the sp variable.
    To solve the system of equations in the Fourier domain,
    target signals and kernels are zero-padded to equal length.
    L: Length of target waveform (duration)
    K: Length of impulse response (duration)
    Mt: Number of target waveforms
    Mi: Number of kernels
    M=Mi=Mt, number of target waveforms must be equal to number of kernels
    :param ks: dict, kernels. (key: observable, val: list of kernel waveforms, one for each speaker)
    :param tg: dict, targets. (key: observable, val: target waveforms)
    :param sp: array, speaker choice: picks Mi (combinations of) kernels,
                e.g. [[1,0],[0,1]] for a signal=speaker mapping in the case of two observables and two speakers.
    :return: array, dim: (K+L) x M, sound that generates target waveforms
    """
    # check whether kernels and targets refer to the same observables
    if tg.keys() != ks.keys():
        raise ValueError("Target waveforms are defined in terms of observables for which no kernels are known.")
    # dict to array
    keys = sorted(tg.keys())
    tgs = np.array([tg[key] for key in keys]).T  # L x Mt
    irs = np.array([np.dot(sp, ks[key]) for key in keys]).T  # K x Mi x Mt
    # zero padding for equal duration
    tgs_pad = np.zeros((len(tgs) + len(irs), *tgs.shape[1:]))  # (K+L) x Mt
    tgs_pad[:-len(irs)] = tgs  # zeros at #todo better to pad at beginning or end?
    irs_pad = np.zeros((len(tgs) + len(irs), *irs.shape[1:]))  # (K+L) x Mi x Mt
    irs_pad[:len(irs)] = irs  # zeros at end
    # fourier domain
    tgs_pad_f = np.fft.fft(tgs_pad.T).T  # (K+L) x Mt
    irs_pad_f = np.fft.fft(irs_pad.T).T  # (K+L) x Mi x Mt
    # solve
    if irs_pad_f.ndim == 3:
        irs_pad_f = np.swapaxes(irs_pad_f, 1, 2)
    sg_pad_f = np.linalg.solve(irs_pad_f, tgs_pad_f)  # (K+L) x Mi
    # transform back
    sg_pad = np.real(np.fft.ifft(sg_pad_f.T).T)  # (K+L) x Mi
    sound = np.dot(sg_pad, sp)  #  (K+L) x Number of connected speakers
    return sound


In [ ]:
# @title Visualization Library

### VISUALIZATION ###
def showvid(fn, width=600):
  with open(fn,'rb') as fid: 
      mp4_base64 = b64encode(fid.read()).decode()
  return HTML(f'<video width={width} controls><source src=data:video/mp4;base64,{mp4_base64} type="video/mp4"></video>')

def writevid(fn, data, fps, arrlim=None, allow_resize=True):
  arr = data.copy()

  if allow_resize:
    n, h, w = arr.shape
    arr_rs = []
    for a in arr:
      arr_rs.append(cv2.resize(a,  (512*w//h, 512), interpolation= cv2.INTER_NEAREST))#cv2.INTER_LINEAR
    arr = np.array(arr_rs)
    
  #arr = arr.transpose(0,2,1)
  if arrlim is None:
    arrlim = 1*abs(arr).max()
    print(f"Set automatic colorscale limits: {arrlim}")
  arr /= arrlim
  kargs = { 'fps': fps, 'quality': 9} #'macro_block_size': 4, 
        #'ffmpeg_params': ['-s','600x450'] }
  imageio.mimwrite(fn, np.uint8(np.clip(arr*255/2+128, 0, 255)),'FFMPEG', **kargs)
  #imageio.mimwrite(fn, np.uint8(cp.clip(arr*255/2+128, 0, 255)), fps=fps, quality=9)

def quiver(p,v0,v1,tstart=0,tstop=-1,tsub=1, ssub=1,fn="./quiver.mp4", plim=None):
  #todo, quiver can't deal with all zero velocity vectors, don't start at beginning
  first_signal = int(np.argwhere(np.sum(abs(p),axis=(1,2)) > 1e-25)[0])
  if tstart < first_signal:
    tstart = first_signal
    print(f"Simulation starts at frame {tstart}.")
  # sound intensity animation
  q0 = p*v0
  q1 = p*v1
  q01 = np.sqrt(q0**2+q1**2)
  q0 /= q01 #unit vectors
  q1 /= q01

  crd0,crd1 = np.meshgrid(range(q0[0].shape[1]), range(q0[0].shape[0]))
  crd0 = crd0[::ssub,::ssub]
  crd1 = crd1[::ssub,::ssub]
  q0 = q0[tstart:tstop:tsub,::ssub,::ssub]
  q1 = q1[tstart:tstop:tsub,::ssub,::ssub]
  q01= q01[tstart:tstop:tsub,::ssub,::ssub]
  lq01 = np.log10(q01)

  ptmp = p[tstart:tstop:tsub]
  if plim is None:
    plim = 0.05*abs(ptmp).max() #limit colormap to fraction of the highest amplitude in selection
    print(f"Set automatic pressure colorscale limits: {plim} Pa")

  fig, ax = plt.subplots(figsize=(16,8))
  ax.set_aspect('equal'), ax.margins(0)
  I = ax.imshow(ptmp[0], cmap='gray', vmin = -plim, vmax=plim,interpolation='spline16') 
  fig.colorbar(I, label="p (Pa)",extend='both')
  Q = ax.quiver(crd0,crd1,q0[0],q1[0],lq01[0],minshaft=2,pivot='mid',clim=[-12,np.nanmax(lq01)])
  fig.colorbar(Q, label="log10(p*v)", extend='min')
  fig.tight_layout()

  def func(t, Q, I):
      I.set_array(ptmp[t])
      Q.set_UVC(q0[t],q1[t],lq01[t])
      return Q,I

  anim = ani.FuncAnimation(fig, func, fargs=(Q,I), frames=len(q0))
  anim.save(fn, writer=ani.writers['ffmpeg'](fps=len(q0)//30, bitrate=1800), dpi=200)

def quiver_acc(p,a0,a1,tstart=0,tstop=-1,tsub=1, ssub=1,fn="./quiver.mp4", plim=None):
  #todo, quiver can't deal with all zero velocity vectors, don't start at beginning
  first_signal = int(np.argwhere(np.sum(abs(p),axis=(1,2)) > 1e-25)[0])
  if tstart < first_signal:
    tstart = first_signal
    print(f"Simulation starts at frame {tstart}.")
  # sound intensity animation
  q0 = p*a0
  q1 = p*a1
  q01 = np.sqrt(q0**2+q1**2)
  q0 /= q01 #unit vectors
  q1 /= q01

  crd0,crd1 = np.meshgrid(range(q0[0].shape[1]), range(q0[0].shape[0]))
  crd0 = crd0[::ssub,::ssub]
  crd1 = crd1[::ssub,::ssub]
  q0 = q0[tstart:tstop:tsub,::ssub,::ssub]
  q1 = q1[tstart:tstop:tsub,::ssub,::ssub]
  q01= q01[tstart:tstop:tsub,::ssub,::ssub]
  lq01 = np.log10(q01)

  ptmp = p[tstart:tstop:tsub]
  if plim is None:
    plim = 0.05*abs(ptmp).max() #limit colormap to fraction of the highest amplitude in selection
    print(f"Set automatic pressure colorscale limits: {plim} Pa")

  fig, ax = plt.subplots(figsize=(16,8))
  ax.set_aspect('equal'), ax.margins(0)
  I = ax.imshow(ptmp[0], cmap='gray', vmin = -plim, vmax=plim,interpolation='spline16') 
  fig.colorbar(I, label="p (Pa)",extend='both')
  Q = ax.quiver(crd0,crd1,q0[0],q1[0],lq01[0],minshaft=2,pivot='mid',clim=[-12,np.nanmax(lq01)])
  fig.colorbar(Q, label="log10(p*a)", extend='min')
  fig.tight_layout()

  def func(t, Q, I):
      I.set_array(ptmp[t])
      Q.set_UVC(q0[t],q1[t],lq01[t])
      return Q,I

  anim = ani.FuncAnimation(fig, func, fargs=(Q,I), frames=len(q0))
  anim.save(fn, writer=ani.writers['ffmpeg'](fps=len(q0)//30, bitrate=1800), dpi=200)

def quiver_dis(p,qx,qy,tstart=0,tstop=-1,tsub=1, ssub=1,fn="./quiver.mp4", plim=None):
  #todo, quiver can't deal with all zero velocity vectors, don't start at beginning
  first_signal = int(np.argwhere(np.sum(abs(p),axis=(1,2)) > 1e-25)[0])
  if tstart < first_signal:
    tstart = first_signal
    print(f"Simulation starts at frame {tstart}.")
  # sound intensity animation
  q0 = p*qx
  q1 = p*qy
  q01 = np.sqrt(q0**2+q1**2)
  q0 /= q01 #unit vectors
  q1 /= q01

  crd0,crd1 = np.meshgrid(range(q0[0].shape[1]), range(q0[0].shape[0]))
  crd0 = crd0[::ssub,::ssub]
  crd1 = crd1[::ssub,::ssub]
  q0 = q0[tstart:tstop:tsub,::ssub,::ssub]
  q1 = q1[tstart:tstop:tsub,::ssub,::ssub]
  q01= q01[tstart:tstop:tsub,::ssub,::ssub]
  lq01 = np.log10(q01)

  ptmp = p[tstart:tstop:tsub]
  if plim is None:
    plim = 0.05*abs(ptmp).max() #limit colormap to fraction of the highest amplitude in selection
    print(f"Set automatic pressure colorscale limits: {plim} Pa")

  fig, ax = plt.subplots(figsize=(16,8))
  ax.set_aspect('equal'), ax.margins(0)
  I = ax.imshow(ptmp[0], cmap='gray', vmin = -plim, vmax=plim,interpolation='spline16') 
  fig.colorbar(I, label="p (Pa)",extend='both')
  Q = ax.quiver(crd0,crd1,q0[0],q1[0],lq01[0],minshaft=2,pivot='mid',clim=[-12,np.nanmax(lq01)])
  fig.colorbar(Q, label="log10(p*x)", extend='min')
  fig.tight_layout()

  def func(t, Q, I):
      I.set_array(ptmp[t])
      Q.set_UVC(q0[t],q1[t],lq01[t])
      return Q,I

  anim = ani.FuncAnimation(fig, func, fargs=(Q,I), frames=len(q0))
  anim.save(fn, writer=ani.writers['ffmpeg'](fps=len(q0)//30, bitrate=1800), dpi=200)

def plot_mesh(volume_shape, dx, rho, c, p_emitters=None, p_sensors=None, outpath=None):
  plt.rc('font', size=16)
  plt.rc('axes', labelsize=16)  # fontsize of the x and y labels
  plt.rc('xtick', labelsize=12)  # fontsize of the tick labels
  plt.rc('ytick', labelsize=12)

  sz = max(int(volume_shape.max() / 50), 1)
  fig, ax = plt.subplots(2, figsize=(10, 10))
  ax[0].imshow(np.mean(c, axis=2).T[::-1]), ax[0].set_title("mean projection along z"), ax[
      0].set_aspect('equal')
  ax[1].imshow(np.mean(c, axis=1).T[::-1]), ax[1].set_title("mean projection along y"), ax[
      1].set_aspect('equal')
  if p_emitters is not None:
    if np.ndim(p_emitters)>1:
      for x, y, z in p_emitters:
          ax[0].add_patch(Circle((x, y), radius=1 * sz, alpha=.2, color='r'))
          ax[1].add_patch(Circle((x, z), radius=1 * sz, alpha=.2, color='r'))
    else:
      x, y, z = p_emitters
      ax[0].add_patch(Circle((x, y), radius=1 * sz, alpha=.2, color='r'))
      ax[1].add_patch(Circle((x, z), radius=1 * sz, alpha=.2, color='r'))

  if p_sensors is not None:
    if np.ndim(p_sensors)>1:
      for x, y, z in p_sensors:
          ax[0].add_patch(Circle((x, y), radius=.7 * sz, alpha=.2, color='b'))
          ax[1].add_patch(Circle((x, z), radius=.7 * sz, alpha=.2, color='b'))
    else:
        x, y, z = p_sensors
        ax[0].add_patch(Circle((x, y), radius=.7 * sz, alpha=.2, color='b'))
        ax[1].add_patch(Circle((x, z), radius=.7 * sz, alpha=.2, color='b'))
  fig.tight_layout()
  plt.show()
  if outpath is not None:
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    fig.savefig(os.path.join(outpath, "fdtd_setup.png"), dpi=300)


def plot_timeseries(data, sr, title=None, lim=None):
    """
    Waveform subplot for each channel
    """
    nsp = data.shape[0]
    nch = 1
    if data.ndim > 1:
        nch = data.shape[1]
    fig = plt.figure(figsize=(24 / 2.54, 6 * nch / 2.54))
    gs = gridspec.GridSpec(nch, 1)
    # plot
    for i in range(nch):
        if data.ndim > 1:
            ch = data[:, i]
        else:
            ch = data
        ax = fig.add_subplot(gs[i, 0])
        ax.plot(1000. * np.arange(nsp, dtype=float) / sr, ch, color='k', lw=1)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.set_xlabel('Time [ms]')
        if lim is not None:
          ax.set_ylim([-lim, lim])
    fig.suptitle(title)
    
def plot_timeseries_grid(data, sr, pth, title=None, lim=None, wav=None, offset_ms=None):
    """
    Waveform subplot for each grid point
    """
    nsp = data.shape[0]
    ny = data.shape[1]
    nx = data.shape[2]
    fig = plt.figure(figsize=(48 / 2.54, 48 * nx / 2.54 / ny))
    gs = gridspec.GridSpec(ny, nx)
    # plot
    for i in range(nx):
        for j in range(ny):
            ch = data[:,j,i]
            ax = fig.add_subplot(gs[j, i])
            ax.plot(1000. * np.arange(nsp, dtype=float) / sr, ch, color='k', lw=.5)
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)
            ax.set_xlabel('Time [ms]')
            if lim is not None:
              ax.set_ylim([-lim, lim])
            ax.set_title(f"x={i}, y={j}")
            if wav is not None:
                t = 1000. * np.arange(len(wav), dtype=float) / sr
                if offset_ms is not None:
                    t += offset_ms
                ax.plot(t, wav, color='#ff7f0e', alpha=.8, lw=.5)
    fig.suptitle(title, fontsize=8)
    fig.tight_layout()
    fn = os.path.join(pth,title+'.png')
    fig.savefig(fn, dpi=300)

def plot_kernels(kns, sr):
    """
    Plots the generated kernels (they are in SI units), after running make_kernels().
    """
    # all observables
    for key, vals in sorted(kns.items()):
        # all speakers
        lim = 1.1*np.max(abs(np.array(vals)))
        for i, val in enumerate(vals):
            plot_timeseries(val, sr, title = f'kernels_{key}_speaker{i}', lim = lim)

# plot sound conditioning results
def plot_sound_simulation(handle, rec, sd, tg, ks, sr, obs):
    """
    Plots the speaker activations that can produce targets.
    A convolution of the speaker activation with the kernel is run,
    to predict whether the target waveform should be generated by the signal.

    :param handle: str
    :param rec: 
    :param obs: dict, definition of observables to turn recording into observables in SI units
    :return:
    """
    sd = sd.T
    nobs = len(tg)
    fig = plt.figure(figsize=(32 / 2.54, (8 * nobs + 1) / 2.54))
    gs = gridspec.GridSpec(nobs + 1, 1)
    # sound
    ax0 = fig.add_subplot(gs[0, 0])
    t = 1000. * np.arange(sd.shape[1], dtype=float) / sr  # in ms
    for i, s in enumerate(sd):
        ax0.plot(t, s, lw=1, alpha=.6, label=f'signal (a.u.), speaker {i}')
    ax0.spines['right'].set_visible(False), ax0.spines['top'].set_visible(False)
    ax0.set_xlabel(r'Time [$ms$]')
    ax0.set_ylabel(r'Voltage to Amplifier')
    ax0.legend(loc='upper right', fontsize=10)

    # targets
    for i, (key, val) in enumerate(sorted(tg.items())):
        # prediction: convolution of sound with kernel for each speaker
        kernel = np.array(ks[key])
        prediction = np.sum(np.array([np.convolve(sd[j], kernel[j]) for j in range(sd.shape[0])]), axis=0)

        shift = 0 # kernel.shape[1]
        ax = fig.add_subplot(gs[i + 1, 0])
        t = 1000. * np.arange(shift, val.shape[0] + shift, dtype=float) / sr  # in ms
        #ax.plot(t, val, '-.',label="target", lw=1)

        t = 1000. * np.arange(prediction.shape[0], dtype=float) / sr  # in ms
        ax.plot(t, prediction, label="prediction", alpha=.6, lw=1)

        t = 1000. * np.arange(rec.shape[0], dtype=float) / sr  # in ms
        func = obs[key]
        ax.plot(t, func(rec), lw=4, alpha=0.4, c='k', label="measured")

        ax.set_xlabel(r'Time [$ms$]')
        ax.set_ylabel(key)
        ax.legend(loc='upper right', fontsize=10)
        ax.spines['right'].set_visible(False), ax.spines['top'].set_visible(False)
    ax0.set_xlim(ax.get_xlim())
    fig.suptitle(handle)
    # save
    fn = f'soundPred_{handle}.png'
    #fig.savefig(identifier +  fn, dpi=300)
    
# plot sound conditioning results
def plot_sound_targeting_field(pth, handle, yth, xth, sds, tg, ks, rec, sr, vlim= None, alim=None, plim=None,tg_offset=0):
    """
    Plots the speaker activations that can produce targets.
    A convolution of the speaker activation with the kernel is run,
    to predict whether the target waveform should be generated by the signal.

    :param handle: str
    :param rec: 
    :return:
    """
    sd = sds[:,:,yth,xth]
    nobs = len(tg)
    fig = plt.figure(figsize=(32 / 2.54, (8 * nobs + 1) / 2.54))
    gs = gridspec.GridSpec(nobs + 1, 1)
    # sound
    ax0 = fig.add_subplot(gs[0, 0])
    t = 1000. * np.arange(sd.shape[1], dtype=float) / sr  # in ms
    for i, s in enumerate(sd):
        ax0.plot(t, s, lw=1, alpha=.6, label=f'signal (a.u.), speaker {i}')
    ax0.spines['right'].set_visible(False), ax0.spines['top'].set_visible(False)
    ax0.set_xlabel(r'Time [$ms$]')
    ax0.set_ylabel(r'Voltage to Amplifier')
    ax0.legend(loc='upper right', fontsize=10)
    
    # targets
    for i, (key, val) in enumerate(sorted(tg.items())):
        # prediction: convolution of sound with kernel for each speaker
        kernel = np.array(ks[key])[:,yth,xth,:].T # nspeaker x time
        #kernel =  [ks[key][:, yth,xth, ii] for ii in range(4)]
        prediction = np.sum(np.array([np.convolve(sd[j], kernel[j]) for j in range(sd.shape[0])]), axis=0)

        shift = 0 # kernel.shape[1]
        ax = fig.add_subplot(gs[i + 1, 0])
        t = 1000. * np.arange(shift, val.shape[0] + shift, dtype=float) / sr  # in ms
        #ax.plot(t, val, '-.',label="target", lw=1)

        t = 1000. * np.arange(prediction.shape[0], dtype=float) / sr  # in ms
        ax.plot(t, prediction, label="prediction", alpha=.6, lw=1)
        
        tg_shifted = np.pad(tg[key],(tg_offset,0),'constant')
        t = 1000. * np.arange(tg_shifted.shape[0], dtype=float) / sr  # in ms
        ax.plot(t, tg_shifted, label="target, shifted by kernel duration", alpha=.6, lw=1)
        
        if rec is not None:
            t = 1000. * np.arange(rec[key].shape[0], dtype=float) / sr  # in ms
            ax.plot(t, rec[key][:,yth,xth], lw=2, alpha=0.4, c='k', label="measured")

        ax.set_xlabel(r'Time [$ms$]')
        ax.set_ylabel(key)
        ax.legend(loc='upper right', fontsize=10)
        ax.spines['right'].set_visible(False), ax.spines['top'].set_visible(False)
        if key=="pressure":
            if plim is not None:
                ax.set_ylim([-plim,plim])
        else:
            if alim is not None:
                ax.set_ylim([-alim,alim])
    if vlim is not None:
        ax0.set_ylim([-vlim,vlim])
        
    ax0.set_xlim(ax.get_xlim())
    txt = handle + f"_x{xth}_y{yth}"
    fig.tight_layout()
    fig.suptitle(txt)
    # save
    fn = f'{txt}.png'
    fig.savefig(os.path.join(pth,fn), dpi=300)
    
def pfield2obs(pfield, startx, stopx, nx, starty, stopy, ny, rho):
    result = {}
    # turn into observables
    # step sizes
    dx = 1e-2 * (stopx - startx) / (nx - 1)  # start/stop in cm
    dy = 1e-2 * (stopy - starty) / (ny - 1)
    result["pressure"] = pfield  # dim: nsamples, ny, nx, nspeakers
    result["acceleration_y"] = - np.gradient(pfield, axis=1) / dy / rho  # Euler Eq.
    result["acceleration_x"] = - np.gradient(pfield, axis=2) / dx / rho  # Euler Eq.
    return result

# Level4 Setup: IR fields and sound conditioning

In [ ]:
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-03_20-43-58_field\2022-03-03_20-43-58stimset_field.h5"
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-10_19-07-57_field\2022-03-10_19-07-57stimset_field.h5"
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-13_16-12-38_field\2022-03-13_16-12-38stimset_field.h5"
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-14_15-59-01_field\2022-03-14_15-59-01stimset_field.h5" # two opposing pairs  all speakers 3cm away
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-14_16-40-21_field\2022-03-14_16-40-21stimset_field.h5" # two opposing pairs  all speakers 1cm away
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-15_12-00-47_field\2022-03-15_12-00-47stimset_field.h5" # kernel hp at 50Hz
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-18_21-34-01_field\2022-03-18_21-34-01stimset_field.h5" # large tank, 2cmx2cm at top left, without silicone walls
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-18_21-51-41_field\2022-03-18_21-51-41stimset_field.h5" # large tank, 2cmx2cm at top left, with silicone walls
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-18_22-09-29_field\2022-03-18_22-09-29stimset_field.h5" # large tank, 8cmx8cm, without silicone walls
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-22_20-14-45_field\2022-03-22_20-14-45stimset_field.h5" # large tank, 6cmx6cm, without silicone walls, not dealing with zero-target in find_sound correctly
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-23_19-59-34_field\2022-03-23_19-59-34stimset_field.h5" # large tank, 6cmx6cm, without silicone walls, opposing pair slightly closer
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-29_16-54-25_field\2022-03-29_16-54-25stimset_field.h5" # large tank, 6cmx6cm, without silicone walls as the one above but with battery for PA4, no grounding cable in water and proper glass to ground coupling via water
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-03-30_10-33-46_field\2022-03-30_10-33-46stimset_field.h5" # Used for 12 behavior experiment, quality decayed over course, sp0 broke
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-04-15_10-44-41_field\2022-04-15_10-44-41stimset_field.h5" # see expt log
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-04-16_12-17-38_field\2022-04-16_12-17-38stimset_field.h5" # see expt log
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-04-17_11-05-01_field\2022-04-17_11-05-01stimset_field.h5"
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-04-20_23-44-24_field\2022-04-20_23-44-24stimset_field.h5" # first time sound targeting with boundary conditions in find_sound_bc
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-04-27_13-09-09_field\2022-04-27_13-09-09stimset_field.h5"
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-05-02_14-45-35_field\2022-05-02_14-45-35stimset_field.h5" # first time with all four being  large speakers 6cmx6cm, r=3cm
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-05-03_08-56-48_field\2022-05-03_08-56-48stimset_field.h5" # putting sp2 (top) further away 3cmx3cm, r=4cm, opposing pair 0.0625 weight -> did reduced undesired y motion
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-05-03_11-07-11_field\2022-05-03_11-07-11stimset_field.h5" # r=3.5cm, opposing pair 0.0833 weight
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-05-03_11-45-33_field\2022-05-03_11-45-33stimset_field.h5" #  opposing pair 0.2 weight
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-05-03_12-42-40_field\2022-05-03_12-42-40stimset_field.h5"
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-20_16-57-59_field\2022-06-20_16-57-59stimset_field.h5" # small speaker 2x2 wall, 3cm monopole target distance -> 3x3 single speaker too pressure target not met
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-20_17-36-20_field\2022-06-20_17-36-20stimset_field.h5" # small speaker 2x2 wall, 6cm monopole target distance -> 3x3 single speaker too pressure target not met even worse (due to too low acceleration target)
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-20_18-07-53_field\2022-06-20_18-07-53stimset_field.h5" # small speaker 2x2 wall, 2.5cm monopole target distance 5x5 grid many sounds 1.5 [1, .1, .1]
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-21_19-19-01_field\2022-06-21_19-19-01stimset_field.h5" # small speaker 2x2 wall, 6cm monopole target distance -> 5x5 grid, 3 [1, 0.1, 0.1] 
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-22_14-19-52_field\2022-06-22_14-19-52stimset_field.h5" #left right: single small speaker, top/bottom: speaker wall 4cm away from inner tank wall -> 3x3 grid
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-22_15-36-47_field\2022-06-22_15-36-47stimset_field.h5" #left right: single small speaker, top/bottom: speaker wall 1cm away -> 3x3 grid
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-22_17-10-27_field\2022-06-22_17-10-27stimset_field.h5" #left right: single small speaker, top/bottom: speaker wall 1cm away -> 5x5 grid all tones ,  bc_tolerance=3, relative_weights=[1, .2, .2], bc_tolerance=5, relative_weights=[1, 1, 1]
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_15-40-57_field\2022-06-23_15-40-57stimset_field.h5" #left right: single small speaker, top/bottom: each 2speakers sidebyside 1cm away -> 3x3 grid
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_17-28-12_field\2022-06-23_17-28-12stimset_field.h5" #left right: single small speaker, top/bottom: each 2speakers sidebyside 1cm away -> 3x3 grid, left/right very close 
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_18-32-55_field\2022-06-23_18-32-55stimset_field.h5"  #left right: single small speaker, top/bottom: each 2speakers sidebyside 1cm away -> 3x3 grid, all .8cm away 5cm source
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_19-33-34_field\2022-06-23_19-33-34stimset_field.h5"  #left right: single small speaker, top/bottom: each 2speakers sidebyside 1cm away -> 3x3 grid, all .8cm away  4cm source
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_20-09-24_field\2022-06-23_20-09-24stimset_field.h5" # all single speakers 3x3
#fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-23_21-20-07_field\2022-06-23_21-20-07stimset_field.h5" # all single small speakers 5x5, bc_tolerance=3, relative_weights=[1, .2, .2], bc_tolerance=5, relative_weights=[1, 1, 1], 'distance2monopole': 0.04
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-25_11-03-47_field\2022-06-25_11-03-47stimset_field.h5" # all single small speakers 5x5,  bc_tolerance=5, relative_weights=[1, .2, .2], bc_tolerance=5, relative_weights=[1, 1, 1], 'distance2monopole': 0.03 GOOD! try louder
fn = r"C:\Users\jlab\Documents\Repos\vocloc_recorder\misc\sound_targeting\2022-06-27_12-07-54_field\2022-06-27_12-07-54stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-02-28_15-47-13_field/2023-02-28_15-47-13stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-06_18-52-12_field/2023-03-06_18-52-12stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-07_13-08-14_field/2023-03-07_13-08-14stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-07_14-32-14_field/2023-03-07_14-32-14stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-07_18-20-10_field/2023-03-07_18-20-10stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-13_field/2023-03-13_irfield.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-13_12-37-23_field/2023-03-13_12-37-23stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-17_21-13-11_field/2023-03-17_21-13-11stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-18_16-51-56_field/2023-03-18_16-51-56stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_10-51-36_field/2023-03-22_10-51-36stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_12-17-58_field/2023-03-22_12-17-58stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_13-06-39_field/2023-03-22_13-06-39stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_16-03-33_field/2023-03-22_16-03-33stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-18_16-51-56_field/2023-03-18_16-51-56stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-17_21-13-11_field/2023-03-17_21-13-11stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_17-40-15_field/2023-03-22_17-40-15stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-22_19-26-12_field/2023-03-22_19-26-12stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-23_11-59-12_field/2023-03-23_11-59-12stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-23_18-57-01_field/2023-03-23_18-57-01stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-03-30_19-22-59_field/2023-03-30_19-22-59stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-17_12-23-33_field/2023-04-17_12-23-33stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-17_13-13-56_field/2023-04-17_13-13-56stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-17_16-54-30_field/2023-04-17_16-54-30stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-18_08-49-48_field/2023-04-18_08-49-48stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-19_09-53-02_field/2023-04-19_09-53-02stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-19_14-03-46_field/2023-04-19_14-03-46stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-19_18-09-45_field/2023-04-19_18-09-45stimset_field.h5"
fn = "/home/jlab-pc09/Documents/data/recordings/202304DireHearingPosCL_ztrack_drop/audio/2023-04-20_17-18-01_field/2023-04-20_17-18-01stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/data/recordings/202304DireHearingPosCL_ztrack_drop/audio/2023-04-20_17-18-01_field/2023-04-21_18-28-36stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-23_09-56-01_field/2023-04-23_09-56-01stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-04-23_18-38-28_field/2023-04-23_18-38-28stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-16_15-09-46_field_smalltank_7cm_waterHeight_bad/2023-05-16_15-09-46stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-17_11-17-56_field/2023-05-17_11-17-56stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-17_15-28-08_field/2023-05-17_15-28-08stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-19_15-56-37_field/2023-05-19_15-56-37stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_10-56-50_field/2023-05-22_11-35-37stimset_field.h5"
#fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_13-50-40_field/2023-05-22_13-50-40stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_14-34-09_field/2023-05-22_14-34-09stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_15-18-50_field/2023-05-22_15-18-50stimset_field_stimonly.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_18-32-58_field/2023-05-22_18-32-58stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-22_21-30-24_field/2023-05-22_21-30-24stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-24_20-00-46_field/2023-05-24_20-00-46stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-05-30_22-46-32_field/2023-05-30_22-46-32stimset_field.h5"
fn = "/home/jlab-pc09/Documents/data/recordings/230305DireHearingPosCL_shallow_thinwalls_drop/audio/2023-06-04_23-50-58_field/2023-06-05_21-56-28stimset_field.h5"
fn= "/home/jlab-pc09/Documents/data/recordings/230305DireHearingPosCL_shallow_thinwalls_drop/audio/2023-06-12_17-51-34_field/2023-06-12_22-48-25stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-08-01_23-24-09_field/2023-08-01_23-24-09stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-08-07_10-32-34_field/2023-08-07_10-32-34stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-08-07_13-16-15_field/2023-08-07_13-16-15stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-08-15_21-56-18_field/2023-08-15_21-56-18stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-10-13_11-36-40_field/2023-10-13_11-36-40stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2023-10-13_15-46-45_field/2023-10-13_15-46-45stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2024-01-03_17-32-16_field/2024-01-03_17-32-16stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2024-01-04_15-04-26_field/2024-01-04_15-04-26stimset_field.h5"
fn = "/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/2024-01-04_17-49-14_field/2024-01-04_17-49-14stimset_field.h5"
name = '2024-01-12_19-30-27_field/2024-01-12_19-30-27stimset_field.h5'
name = '2024-01-19_14-11-08_field/2024-01-19_14-11-08stimset_field_stimonly.h5'
name = '2024-01-19_14-11-08_field/2024-01-23_17-46-17stimset_field.h5'
fn = os.path.join('/home/jlab-pc09/Documents/repos/vocloc_recorder/misc/sound_targeting/',name)
out = fn[:-16] + '_'
base_path = os.path.split(fn)[0]
session =  os.path.split(out)[1]
d = load_from_h5(fn, verbose=0)
if 'stimset_field.h5' in fn:
    pm = d['test0_params']
    sr = d['globalconfig']['audio']['rate']
print(fn,out,base_path, sep='\n')
d.keys()

In [ ]:
#sr = 51200.0

## Content overview

In [ ]:
sr

In [ ]:
d['globalparams']

In [ ]:
d['irfield'].shape #nsamples, ny, nx, nspeaker, repetitions

In [ ]:
d['irfield_params']

In [ ]:
d['kernelfield'].keys()

In [ ]:
d['kernelfield']['pressure'].shape #nsamples, ny, nx, nspeaker 

In [ ]:
d['wavs'].keys()

In [ ]:
list(d['wavs'].values())[0].shape

In [ ]:
d['sounds_tg'].keys()

In [ ]:
list(d['sounds_tg'].values())[0] 

In [ ]:
list(d['sounds_tg'].values())[0]['target']

In [ ]:
list(d['sounds_tg'].values())[0]['sp_map']

In [ ]:
list(d['sounds_tg'].values())[0]['target']['pressure'].shape # waveform

In [ ]:
d['sounds_tg_params']

In [ ]:
d['sounds_cdmap0'].keys()

In [ ]:
for key, sd in d['sounds_cdmap0'].items():
    print(sd.shape)

In [ ]:
print(*list(d['sounds_cdmap0'].keys()), sep='\n')

In [ ]:
list(d['sounds_cdmap0'].values())[0].shape # sound field, nspeaker, duration, ny, nx

In [ ]:
d['test0'].keys()

In [ ]:
list(d['test0'].values())[2].shape

In [ ]:
d['sounds_tg_params']['p_amplitude']

## Plots and Video exports

In [ ]:
# Visualize fields
sp = 0
plimit = 500#250 # pressure limit
acclimit = 10 # acceleration limit
vlim = 4
initial_ms = 30

### Impulse responses

In [ ]:
# ir plot for all speakers and positions
sp = 1
ir = d['irfield'][:,:,:,sp,:].mean(-1)
#ir = ir.reshape(len(ir),-1)
#plot_timeseries(ir, sr, title=None, lim=plimit)

In [ ]:
ir.shape

In [ ]:
plt.figure()
powerSpectrum, frequenciesFound, time, imageAxis = plt.specgram(ir[:,2,2], Fs=sr)

In [ ]:
from pylab import *
def spectrum(x, sr):
    dt = 1/sr                                # Define the sampling interval.
    N = x.shape[0]                           # Define the total number of data points.
    T = N * dt                               # Define the total duration of the data.

    #x  = hanning(N) * x                      # Apply the Hanning taper to the data.
    xf = fft(x - x.mean())                   # Compute Fourier transform of x.
    Sxx = 2 * dt ** 2 / T * (xf * conj(xf))  # Compute the spectrum,
    Sxx = real(Sxx[:int(len(x) / 2)])        # ... and ignore negative frequencies.

    df = 1 / T.max()                         # Determine the frequency resolution.
    fNQ = 1 / dt / 2                         # Determine the Nyquist frequency.
    faxis = arange(0,fNQ,df)                 # Construct a frequency axis.
    return faxis[1:], 10*log10(Sxx[1:])

plt.figure()
f, p = spectrum(ir[:,2,2], sr)
plot(f,p)    # Plot spectrum vs frequency,
xlim([f[0], 10000])                    # Select frequency range,
#ylim([-40, 20])                          # ... and the power range.
xlabel('Frequency [Hz]')                 # Label the axes
ylabel('Power [$V^2$/Hz]')

In [ ]:
def fftcoefs(x, sr):
    dt = 1/sr                                # Define the sampling interval.
    N = x.shape[0]                           # Define the total number of data points.
    T = N * dt                               # Define the total duration of the data.
    #x  = hanning(N) * x                      # Apply the Hanning taper to the data.
    return rfftfreq(len(x), d = dt), abs(rfft(x))

plt.figure()
f, p = fftcoefs(ir[:,2,2], sr)
plot(f,p)    # Plot spectrum vs frequency,
xlim([20, 1500])                    # Select frequency range,
ylim([0, 700])                          # ... and the power range.
xlabel('Frequency [Hz]')                 # Label the axes
ylabel('Coeff [$V$/Hz]')

In [ ]:
irs = d['irfield'].mean(-1)
plt.figure()
for sptmp in range(irs.shape[-1]):
    f, p = fftcoefs(irs[:,2,2,sptmp], sr)
    plot(f,p,label=f"speaker {sptmp}")    # Plot spectrum vs frequency,
    xlim([20, 4000])                    # Select frequency range,
    ylim([0, 700])                          # ... and the power range.
    xlabel('Frequency [Hz]')                 # Label the axes
    ylabel('Coeff [$V$/Hz]')
plt.legend()
plt.savefig(os.path.join(base_path,"fftcoef.png"),dpi=200)

In [ ]:
base_path

In [ ]:
irs = d['irfield'].mean(-1)
plt.figure()
sptmp = 0
for yth in range(5):
    for xth in range(5):
        f, p = fftcoefs(irs[:,yth,xth,sptmp], sr)
        plot(f,p, alpha=.5)    # Plot spectrum vs frequency,
        xlim([20, 1500])                    # Select frequency range,
        ylim([0, 600])                          # ... and the power range.
        xlabel('Frequency [Hz]')                 # Label the axes
        ylabel('Coeff [$V$/Hz]')
plt.savefig(os.path.join(base_path,"fftcoef2.png"),dpi=200)

In [ ]:
# ir video (p)
for sp in range(4):
    fntmp =  out + f"IR_pressure_speaker{sp}.mp4"
    sig = d['irfield'][:,:,:,sp,:].mean(-1)
    writevid(fntmp, sig, len(sig)//30, arrlim = plimit)

### Kernels

In [ ]:
# kernel plot
sp = 0
for k, v in d['kernelfield'].items():
    if k=="pressure":
        lim = plimit
    else:
        lim = acclimit
    sig =v[:,:,:,sp]
    title = f"{session}KERNEL_speaker{sp}_{k}"
    plot_timeseries_grid(sig, sr, pth=base_path, title=title, lim=lim)

In [ ]:
# kernel video  (p, ax, ay)
sp = 0
for k, v in d['kernelfield'].items():
    fntmp =  out + f"KERNEL_{k}_speaker{sp}.mp4"
    if k=="pressure":
        lim = plimit
    else:
        lim = acclimit
    sig =v[:,:,:,sp]
    writevid(fntmp, sig, len(sig)//30, arrlim = lim)

In [ ]:
# kernel quiver (p, p*v)
sp = 0
p = d['kernelfield']['pressure'][:,:,:,sp]
ax = d['kernelfield']['acceleration_x'][:,:,:,sp]
ay = d['kernelfield']['acceleration_y'][:,:,:,sp]
vx = np.trapz(ax,axis=0,dx=1/sr)
vy = np.trapz(ay,axis=0,dx=1/sr)
qx = np.trapz(vx,axis=0,dx=1/sr)
qy = np.trapz(vy,axis=0,dx=1/sr)
quiver(p,vx,vy,tstart=0,tstop=-1,tsub=5, ssub=1,fn=out + f"KERNEL_speaker{sp}_pv_quiver.mp4", plim=plimit)
quiver_acc(p,ax,ay,tstart=0,tstop=-1,tsub=5, ssub=1,fn=out + f"KERNEL_speaker{sp}_pa_quiver.mp4", plim=plimit)
quiver_dis(p,qx,qy,tstart=0,tstop=-1,tsub=5, ssub=1,fn=out + f"KERNEL_speaker{sp}_px_quiver.mp4", plim=plimit)

### Test recordings
PLOTS  
- sound targeting plot

VIDEOS  
- unconditioned stimuli  
- sound targeting for pos x,y
- artificial field where at each position the measurement for the targeted sound is used  

In [ ]:
# sound targeting plot
xth = 1
yth = 2
for key in sorted(d['sounds_tg'].keys()):
    sds = d['sounds_cdmap0'][key]
    tg = d['sounds_tg'][key]['target']
    ks = d['kernelfield']
    pfield = d['test0'][key][:,:,:,:,yth,xth].mean(3)
    pm = d['test0_params']
    rec = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
    plot_sound_targeting_field(base_path, session+"TEST_"+key, yth, xth, sds, tg, ks, rec, sr, vlim=vlim, alim=acclimit, plim=plimit, tg_offset=0)

In [ ]:
keys_raw =  sorted([k for k in d['test0'].keys() if "Conditioned" not in k]) # nsamples, ny, nx, repeats
keys_with_cond = sorted([k for k in d['test0'].keys() if "Conditioned" in k]) # nsamples, ny, nx, repeats, ny(target), nx(target)

In [ ]:
keys_raw

In [ ]:
keys_with_cond

In [ ]:
vDAQ_2_vAS1 = 1 / 10 ** (50 / 20) 
vAS1_2_p = 1 / (40e-6)
m2p =  vDAQ_2_vAS1 * vAS1_2_p

In [ ]:
405/m2p

In [ ]:
m2p*5

In [ ]:
key = keys_raw[0]
key

In [ ]:
keysame = keys_with_cond[0]
keysame

In [ ]:
d['test0'][key].shape

In [ ]:
d['test0'][keysame].shape

In [ ]:
pfield = d['test0'][key][:,:,:,:,0].mean(3) 
obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
tgp = d['sounds_tg'][keysame]['target']['pressure']
offset = d['sounds_tg_params']['smp_horizon']

In [ ]:
def a_sphere(p, r, sr, rho, c):
    """
    see e.g. Eq A3 in https://doi.org/10.1242/jeb.093831, v = (1+i/kr)*p/(rho*c)
    :param p:
    :param r:
    :return:
    """
    # radial acceleration from pressure for a spherical wave
    k = 2 * np.pi * np.fft.fftfreq(p.shape[-1])[1:] * sr / c # exclude DC component (divergence of 1/kr)
    coeff = 1 + 1j / (k * r)
    derivative = -2 * np.pi * 1j * np.fft.fftfreq(p.shape[-1])[1:] * sr
    tmp = np.fft.fft(p)[1:] * coeff * derivative
    a = np.real(np.fft.ifft(np.insert(tmp, 0, 0) / (rho*c)))  # insert zero DC
    return a

In [ ]:
obs['pressure'].shape

In [ ]:
# first find pressure ampltidue
plt.figure()
plt.plot(obs['pressure'][:,2,2])
plt.plot(.5*tgp[offset:])

In [ ]:
plt.figure()
plt.plot(obs['acceleration_x'][:,2,2])
r = 0.03
plt.plot(a_sphere(0.5*tgp[offset:],r,sr,d['globalparams']['rho'],d['globalparams']['c']))

In [ ]:
peakax = abs(d['kernelfield']['acceleration_x'][:,2,2,0]).max()
peakp = abs(d['kernelfield']['pressure'][:,2,2,0]).max()
kratio = peakp/peakax
ratio_list = []
rs =  np.linspace(0.01,0.15,100)
for r in rs:
    p = tgp[offset:]
    ratio_list.append(abs(p).max()/abs(a_sphere(p,r,sr,d['globalparams']['rho'],d['globalparams']['c'])).max())
plt.figure()
plt.axhline(kratio, label="Measured from kernel")
plt.xlabel("Distance from monopole (cm)")
plt.ylabel("peak pressure / peak x-acceleration")
plt.plot(100*rs, ratio_list,c='k', label = "Monopole theory")
plt.legend()

In [ ]:
peakax = abs(d['kernelfield']['acceleration_x'][:,1,1,1]).max()
peakp = abs(d['kernelfield']['pressure'][:,1,1,1]).max()
kratio = peakp/peakax
kratio

In [ ]:
plt.figure()
rs = np.linspace(0.001,1,1000)
ratios = []
for r in rs:
    ratios.append(abs(d['kernelfield']['pressure'][:,1,1,1]).max()/abs(a_sphere(d['kernelfield']['pressure'][:,1,1,1],r=r, sr=sr,rho=1000,c=1500)).max())
plt.plot(rs,ratios)

In [ ]:
d['kernelfield']['pressure'].shape

In [ ]:
# in a tank the p/a ratio does not change with distance to the sound source -> todo: show how this holds for high frequencies
ratio_list = []
rs =  np.linspace(0.01,0.15,100)
plot_monopole_theory = 1

if plot_monopole_theory:
    for r in rs:
        idx = 0
        #for idx in idx_measured:
        p = d['kernelfield']['pressure'][:,2,idx,0]
        ratio_list.append((p/abs(a_sphere(p,r,sr,d['globalparams']['rho'],d['globalparams']['c']))).max())
    
    
idx_measured = np.arange(0,3)
d_idx_measured = 0.015 * idx_measured + 0.08 #distance of first measure point to speaker
kratios = []
for idx in idx_measured:
    peakax = abs(d['kernelfield']['acceleration_x'][:,2,idx,0]).max()
    peakp = abs(d['kernelfield']['pressure'][:,2,idx,0]).max()
    kratio = peakp/peakax
    kratios.append(kratio)

plt.figure()
plt.plot(100*d_idx_measured,kratios, label="Measured from kernel")
plt.xlabel("Distance from monopole (cm)")
plt.ylabel("peak pressure / peak x-acceleration")
if plot_monopole_theory:
    plt.plot(100*rs, ratio_list,c='k', label = "Monopole theory")
plt.legend()

In [ ]:
# unconditioned stimuli  
initial_ms = 30
for k in keys_raw[:1]:
    for i in range(2):
        fntmp = f"{out}_{k}_speaker_idx{i}"
        pfield = d['test0'][k][:,:,:,:,i].mean(3)[:int(initial_ms*1e-3*sr)] #todo: will have a speaker dimension soon
        print(pfield.shape)
        obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
        vx = np.trapz(obs["acceleration_x"],axis=0,dx=1/sr)
        vy = np.trapz(obs["acceleration_y"],axis=0,dx=1/sr)
        qx = np.trapz(vx,axis=0,dx=1/sr)
        qy = np.trapz(vy,axis=0,dx=1/sr)
        quiver(obs["pressure"],vx,vy,tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_pv_quiver.mp4', plim=plimit)
        quiver_acc(obs["pressure"],obs["acceleration_x"],obs["acceleration_y"],tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_pa_quiver.mp4', plim=plimit)
        quiver_dis(obs["pressure"],qx,qy,tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_px_quiver.mp4', plim=plimit)
        writevid(fntmp+"_pressure.mp4", obs["pressure"], len(obs["pressure"])//30, arrlim = plimit)

In [ ]:
# unconditioned stimuli
initial_ms = 10
k = keys_raw[0]
i = 0
fntmp = f"{out}_{k}_speaker_idx{i}"
pfield = d['test0'][k][:,:,:,:,i].mean(3)[:int(initial_ms*1e-3*sr)]
print(pfield.shape)
obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])

#target waveform
wav, sr = sf.read(r"/home/jlab-pc09/Documents/data/playback/wav_folder/rubberdrop_HP100LP20000_1stDrop_1stpart_normalized_smalldelay_shortenedForDAQ_upsampled51200.wav") 
scale = np.percentile(abs(pfield),99.5)

for ob, v in obs.items():
    if ob=="pressure":
        lim = plimit
        wfm = scale*wav
    else:
        lim = acclimit
        wfm = None
    plot_timeseries_grid(v, sr, pth=base_path, title=fntmp+f"_{ob}", lim=lim, wav=wfm, offset_ms=0.5)

In [ ]:
obs["pressure"].shape

In [ ]:
# sound targeting for pos x,y
xchoose = 2
ychoose = 2
#initial_ms = 30
for k in keys_with_cond[:2]:
    fntmp = f"{out}_...{k[-50:]}_target_for_x{xchoose}y{ychoose}"
    pfield = d['test0'][k][:,:,:,:,ychoose,xchoose].mean(3)[:int(initial_ms*1e-3*sr)]
    print(pfield.shape)
    obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
    vx = np.trapz(obs["acceleration_x"],axis=0,dx=1/sr)
    vy = np.trapz(obs["acceleration_y"],axis=0,dx=1/sr)
    qx = np.trapz(vx,axis=0,dx=1/sr)
    qy = np.trapz(vy,axis=0,dx=1/sr)
    quiver(obs["pressure"],vx,vy,tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_pv_quiver.mp4', plim=plimit/2)
    quiver_acc(obs["pressure"],obs["acceleration_x"],obs["acceleration_y"],tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_pa_quiver.mp4', plim=plimit/2)
    quiver_dis(obs["pressure"],qx,qy,tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_px_quiver.mp4', plim=plimit/2)
    writevid(fntmp+"pressure.mp4", obs["pressure"], len(obs["pressure"])//30, arrlim = plimit)

In [ ]:
# artificial field where at each position the measurement for the targeted sound is used  
for k in keys_with_cond[:1]:
    fntmp = f"{out}_{k}_CLOSEDLOOP"
    pmean = d['test0'][k].mean(3)[:int(initial_ms*1e-3*sr)]
    print(pmean.shape)
    #closed loop targeted playback
    p_cl = np.zeros(pmean.shape[:3])
    ay_cl = np.zeros(pmean.shape[:3])
    ax_cl = np.zeros(pmean.shape[:3])
    for i in range(pm['ny']):
        for j in range(pm['nx']):
            pfield = pmean[:,:,:,i,j] #choose field from sound that was targeted for this grid position
            obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
            p_cl[:,i,j] = obs["pressure"][:,i,j]
            ay_cl[:,i,j] = obs["acceleration_y"][:,i,j]
            ax_cl[:,i,j] = obs["acceleration_x"][:,i,j]
    vx = np.trapz(ax_cl,axis=0,dx=1/sr)
    vy = np.trapz(ay_cl,axis=0,dx=1/sr)
    qx = np.trapz(vx,axis=0,dx=1/sr)
    qy = np.trapz(vy,axis=0,dx=1/sr)
    quiver(obs["pressure"],vx,vy,tstart=0,tstop=-1,tsub=20,ssub=1,fn=fntmp+'_pv_quiver.mp4', plim=plimit)
    quiver_acc(p_cl,ax_cl,ay_cl,tstart=0,tstop=-1,tsub=20,ssub=1,fn=fntmp+'_pa_quiver.mp4', plim=plimit)
    quiver_dis(obs["pressure"],qx,qy,tstart=0,tstop=-1,tsub=5,ssub=1,fn=fntmp+'_px_quiver.mp4', plim=plimit)
    writevid(fntmp+"pressure.mp4",p_cl, len(p_cl)//30, arrlim = plimit)

In [ ]:
# artificial field where at each position the measurement for the targeted sound is used  
#initial_ms = 90
for k in keys_with_cond:
    fntmp = f"{out}_{k}_CLOSEDLOOP"
    pmean = d['test0'][k].mean(3)#[:int(initial_ms*1e-3*sr)]
    print(pmean.shape)
    #closed loop targeted playback
    cl = {}
    cl["pressure"] = np.zeros(pmean.shape[:3])
    cl["acceleration_y"]  = np.zeros(pmean.shape[:3])
    cl["acceleration_x"] = np.zeros(pmean.shape[:3])
    for i in range(pm['ny']):
        for j in range(pm['nx']):
            pfield = pmean[:,:,:,i,j] #choose field from sound that was targeted for this grid position
            obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
            cl["pressure"][:,i,j] = obs["pressure"][:,i,j]
            cl["acceleration_y"][:,i,j] = obs["acceleration_y"][:,i,j]
            cl["acceleration_x"][:,i,j] = obs["acceleration_x"][:,i,j]
    tg = d['sounds_tg'][k]['target']
    for ob, v in tg.items():
        if ob=="pressure":
            lim = plimit
        else:
            lim = acclimit
        print(ob)
        plot_timeseries_grid(cl[ob], sr, pth=base_path, title=fntmp+f"_{ob}", lim=lim, wav=v, offset_ms=0)

In [ ]:
# field for specified position
points = [[0,1],[0,2],[2,2],[3,3]]
for xth,yth in points:
    for k in keys_with_cond:
        print(k)
        fntmp = f"{out}_{k}_TargetedFor_x{xth}_y{yth}"
        pmean = d['test0'][k].mean(3)#[:int(initial_ms*1e-3*sr)]
        #closed loop targeted playback
        pfield = pmean[:,:,:,yth,xth] #choose field from sound that was targeted for this grid position
        obs = pfield2obs(pfield, startx=pm['startx'], stopx=pm['stopx'], nx=pm['nx'], starty=pm['starty'], stopy=pm['stopx'], ny=pm['ny'], rho=d['globalparams']['rho'])
        tg = d['sounds_tg'][k]['target']
        for ob, v in tg.items():
            if ob=="pressure":
                lim = plimit
            else:
                lim = acclimit
            print(ob)
            plot_timeseries_grid(obs[ob], sr, pth=base_path, title=fntmp+f"_{ob}", lim=lim, wav=v, offset_ms=0)

In [ ]:
# max values
sp = 0
abs(d['irfield'][:,:,:,sp,:]).max(-1).max(0)

In [ ]:
#bilinear interpol
#for ypos in range(300):
 #   for xpos in range(300):

ypos = 138
xpos = 124

minx=61
miny=63
gridw=138
gridh=142
ny = 5
nx = 5

ep = 1e-5 # numerical hack to handle cases position is outside grid
epy_grid = (ny-1)*ep/gridh/2 # to handle case between two points, factor 1/2 to not revert outside grid correction  
epx_grid = (nx-1)*ep/gridw/2
ycl = np.clip(ypos, miny+ep, miny+gridh-ep)
xcl = np.clip(xpos, minx+ep, minx+gridw-ep)
#print(ycl,xcl)
# 4 neighbours on grid
y = ycl-miny
x = xcl-minx
y /= gridh 
x /= gridw
y *= (ny - 1)
x *= (nx - 1)
print(y,x)
yupp = int(np.ceil(y+epy_grid))
ylow = int(np.floor(y+epy_grid))
xupp = int(np.ceil(x+epx_grid))
xlow = int(np.floor(x+epx_grid))
neighbours = np.array([[yupp,xupp], #y1,x1
                       [yupp,xlow], #y1,x0
                       [ylow,xupp], #y0,x1
                       [ylow,xlow]  #y0,x0
                      ]).astype(int)
#print(neighbours)

# Bilinear interpolation in metric space (not grid space)
ys = np.round(np.linspace(miny, miny + gridh, ny)).astype(int)
xs = np.round(np.linspace(minx, minx + gridw, nx)).astype(int)
y1 = ys[yupp]
y0 = ys[ylow]
x1 = xs[xupp]
x0 = xs[xlow]
area = (y1-y0)*(x1-x0)
#print(area)
weights = np.array([(ycl-y0)*(xcl-x0),
                    (ycl-y0)*(x1-xcl),
                    (y1-ycl)*(xcl-x0),
                    (y1-ycl)*(x1-xcl)])/area

weights[weights<ep] = 0 #num. hack
weights /= weights.sum()
print(weights)
im = np.zeros((256,256, 3)).astype('uint8')
for xx in xs:
    for yy in ys:
        im = cv2.circle(im, (xx, yy), 8, (255, 255, 255), -1)
# picked position
for i, nb in enumerate(neighbours):
    im = cv2.circle(im, (xs[nb[1]], ys[nb[0]]), 8, (0, int(weights[i]*255), 0), -1)

im = cv2.circle(im, (xpos,ypos), 3, (255, 0, 255), -1)
        #print(neighbours)
plt.figure()
plt.imshow(im)

In [ ]:
y+ep/2

In [ ]:
[[w, yidx, xidx] for w,(yidx, xidx) in zip(weights, neighbours)]

# L4 directional hearing playback tank geometry

In [ ]:
mesh = create_mesh3(with_sensors=False, with_emitters=False)
f = 10e-3/mesh["dx"]
d_emitter = 0.1 # diameter in cm
r = d_emitter/2
emitter_y_center = 24 # in cm
emitter_z_center = 13.5 # in cm
emitters_y, emitters_z = np.meshgrid(f*np.linspace(emitter_y_center-r, emitter_y_center+r, int(f*d_emitter)+1), f*np.linspace(emitter_z_center-r, emitter_z_center+r, int(f*d_emitter)+1))
p_emitters = [[int(f*19),y,z] for (y,z) in zip(emitters_y.flatten().astype(int),emitters_z.flatten().astype(int))]
mesh['p_emitters'] = p_emitters#(((int(f*19), int(f*24), int(f*14)))) # emitter extend
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

In [ ]:
a = {"test":4}
list(a.values())[0]

In [ ]:
fn = r"rubberdrop_HP100LP20000_1stDrop_1stpart_normalized_smalldelay_shortenedForDAQ_upsampled51200.wav"
#urllib.request.urlretrieve('http://people.physik.hu-berlin.de/~anandjoh/media/drop.wav', fn)
wav, sd_sr = sf.read(fn)
wav = np.pad(wav, int(.02 * sd_sr))
n_emitters = len(mesh['p_emitters'])
sd = np.repeat(wav[:, np.newaxis], n_emitters, axis=1)
sd = 23784*sd
sd.shape

In [ ]:
plane_pos=int(f*14)

In [ ]:
# record pressure and velocity in plane
plane_pos=int(f*14)
p_sim, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=plane_pos)

In [ ]:
# pressure video
fn_p =  "./p_sim.mp4"
writevid(fn_p, p_sim, len(p_sim)//30)

In [ ]:
showvid(fn_p,width=600)

### Comparison to measured sound field

In [ ]:
fn = r"C:\Users\jlab\Documents\Johannes\Data\tmp\20220222_1544field.h5" #working speaker
d = load_from_h5(fn, verbose=0)
p_meas = d["p"]
p_meas = p_meas.mean(-1) # across repetitions
p_meas.shape

In [ ]:
# Origin of measurements
meas_originx = int(f*20)
meas_originy = int(f*20)

In [ ]:
startx = d["param"]["startx"]
starty = d["param"]["starty"]
stopx = d["param"]["stopx"]
stopy = d["param"]["stopy"]
nx =  d["param"]["nx"]
ny = d["param"]["ny"]
meas_sr =  d["param"]["samplerate"]
meas_samples_x, meas_samples_y = np.meshgrid(f*np.linspace(startx, stopx, nx), f*np.linspace(starty, stopy, ny))
meas_samples_x += meas_originx
meas_samples_y += meas_originy
meas_samples_x = (meas_samples_x).astype(int)
meas_samples_y = (meas_samples_y).astype(int)
p_sensors = [[x,y,plane_pos] for (x,y) in zip(meas_samples_x.flatten(),meas_samples_y.flatten())]
Sim.mesh["p_sensors"] = p_sensors
plot_mesh(**Sim.mesh,outpath='./')

In [ ]:
p_sim_at_measpos = p_sim[:,meas_samples_y,meas_samples_x]

In [ ]:
p_meas_filt = butter_highpass_filter(p_meas.T, 200, meas_sr, order=4, repeat=4).T
p_sim_at_measpos_filt =  butter_highpass_filter(p_sim_at_measpos.T, 200, meas_sr, order=4, repeat=4).T

In [ ]:
# listen to sounds
import sounddevice as sd
p_normalized = p_meas/abs(p_meas).max()

In [ ]:
for i in range(9):
    for j in range(9):
        print(i,j)
        sd.play(p_normalized[:,i,j])
        #sd.wait()
        time.sleep(.5)

In [ ]:
p_sim_at_measpos.shape

In [ ]:
p_meas.shape

In [ ]:
n_pos = nx*ny
meas_offset = int(0.00058*meas_sr)
fig, ax = plt.subplots(figsize=(5,5))
plt.plot(p_meas_filt[meas_offset:,4,4])
plt.plot(p_sim_at_measpos_filt[:-meas_offset,4,4])
plt.show()

In [ ]:
p_meas_aligned = p_meas_filt[meas_offset:]
p_sim_at_measpos_aligned = p_sim_at_measpos_filt[:-meas_offset]

In [ ]:
4051*p_meas_aligned.max()/p_sim_at_measpos_aligned.max()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
t = np.arange(len(p_meas_aligned))/meas_sr
for i in range(9):
    for j in range(9):
        plt.plot(1e3*t,p_meas_aligned[:,i,j], 'orange', alpha = .8,label="Measurement")
        plt.plot(1e3*t,p_sim_at_measpos_aligned[:,i,j],'k',alpha = .5,label="Simulation")
#plt.legend()
plt.show()

In [ ]:
# Merge simulation and measurement
p_merged = np.hstack((p_meas_aligned,p_sim_at_measpos_aligned))
p_merged.shape

In [ ]:
# pressure video
fn_p =  "./p_measurementLeft_simulationRight.mp4"
writevid(fn_p, p_merged, len(p_merged)//80)

In [ ]:
# pressure video
showvid(fn_p,width=600)

## Example 1

Microscope setup

### Define Mesh

In [ ]:
mesh = create_mesh2()
Sim = WeeFDTD(mesh)
plot_mesh(**mesh)

### Define Sound

In [ ]:
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=10000, pulse_duration=0.001)
sd_2d = np.vstack((sd,sd)).T # Two speakers in phase
plot_timeseries(sd_2d,sd_sr)

### Run Simulation: pressure sensors

In [ ]:
# record pressure at sensors
p_rec = Sim.rec_p_sensors(sd_2d, sd_sr)

In [ ]:
t = np.arange(p_rec.shape[0])/sd_sr
plt.plot(t*1000, p_rec) # a curve for each sensor

### Run Simulation: record plane

In [ ]:
# record pressure and velocity in plane
p_rec, v0, v1 = Sim.rec_pv_plane(sd_2D, sd_sr, plane_axis=1, plane_pos=16)

In [ ]:
# pressure video
fn_p =  "./pressure.mp4"
writevid(fn_p, p_rec, len(p_rec)//30)

In [ ]:
showvid(fn_p,width=600)

In [ ]:
# sound intensity video
# Quiver plot is slow, consider subsampling in time [tsub] and space [ssub].
fn_q = "./quiver.mp4"
quiver(p_rec,v0,v1,tstart=0,tstop=-1,tsub=10,ssub=2,fn=fn_q)

In [ ]:
showvid(fn_q,width=800)

 ## Example 2
 Same with Triangulation setup

### Define Mesh

In [ ]:
mesh = create_mesh1()
Sim = WeeFDTD(mesh)
plot_mesh(**mesh)

In [ ]:
# can add emitters and sensors
Sim.mesh['p_emitters'] = ((200,200,100))
Sim.mesh['p_sensors'] = ((180,200,100),(150,150,100))
plot_mesh(**mesh)

### Define Sound

In [ ]:
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=10000, pulse_duration=0.001)

### Run Simulation: record plane

In [ ]:
# THIS ONE TAKES TIME AND CAN EXCEED YOUR DAILY GPU ALLOWANCE
# record pressure and velocity in plane
# you can only execute this cell a few times before you reach your daily GPU usage limit :P 
p_rec, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=1, plane_pos=200)

In [ ]:
# pressure video
fn_p2 =  "./pressure2.mp4"
writevid(fn_p2, p_rec, len(p_rec)//30)

In [ ]:
showvid(fn_p2,width=600)

## Attenuation over distance

See eg. (http://asa.scitation.org/doi/10.1121/1.1515799)


In [ ]:
mesh = create_mesh3(with_sensors=False, with_emitters=False)
mesh['p_emitters'] = (((16, 24, 15))) # single emitter
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

### Define Sound

In [ ]:
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=10000, pulse_duration=0.002)
plot_timeseries(sd,sd_sr)

### Run Simulation

In [ ]:
# record pressure and velocity in plane
p_rec, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=15)

In [ ]:
# pressure video
fn_p =  "./pressure.mp4"
writevid(fn_p, p_rec, len(p_rec)//30)

In [ ]:
showvid(fn_p,width=600)

In [ ]:
amps_along_axis =  np.max(abs(p_rec[:,:,24]),axis=0)

### Look at decay profile

In [ ]:
plt.figure()
plt.plot(amps_along_axis)

### Loop across frequencies

In [ ]:
print(1500/Sim.mesh['dx']) # f = c/lambda

In [ ]:
0.16/1500 # ~time it takes to reach wall

In [ ]:
1500/2*np.sqrt(0.3**2+0.3**2+0.12**2) # cutoff freq

In [ ]:
# loop over frequencies
sd_sr = Sim.sr
freqs = np.array([20000, 10000, 5000, 1000, 500, 100])
pulse_durations = np.array([max(7/freq,20*0.16/1500) for freq in freqs])
amps = []
for (freq,dur) in zip(freqs, pulse_durations):
  sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=freq, pulse_duration=dur)
  start = int((dur*sd_sr)//3) #cut beginning
  sd = sd[start:] 
  p_rec, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=15)
  # pressure video
  fn_p =  f"./pressure_gp{freq}Hz.mp4"
  writevid(fn_p, p_rec, len(p_rec)//30)
  amps_along_axis =  np.max(abs(p_rec[:,:,24]),axis=0)
  amps.append(amps_along_axis)

In [ ]:
plt.figure(figsize=(12,12))
for i, freq in enumerate(freqs):
  plt.plot(amps[i], label = f"{freq}Hz")
plt.legend()
plt.savefig("pAttenuation.png")

## Attenuation through a bottleneck

In [ ]:
mesh = create_mesh3(with_sensors=False, with_emitters=False)
mesh['p_emitters'] = (((16, 34, 15))) # single emitter
cx,cy,cz = mesh['volume_shape']//2
dhl = 5 # duct half length
dhw = 2 # duct half width
mesh['c'][cx-dhl:cx+dhl,0:cy-dhw,:] = 343
mesh['rho'][cx-dhl:cx+dhl,0:cy-dhw,:] = 1.2
mesh['c'][cx-dhl:cx+dhl,cy+dhw:,:] = 343
mesh['rho'][cx-dhl:cx+dhl,cy+dhw:,:]  = 1.2
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

### Define Sound

In [ ]:
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=10000, pulse_duration=0.002)
plot_timeseries(sd,sd_sr)

### Run Simulation

In [ ]:
# record pressure and velocity in plane
p_rec, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=15)

In [ ]:
# pressure video
fn_p =  "./pressure.mp4"
writevid(fn_p, p_rec, len(p_rec)//30)

In [ ]:
showvid(fn_p,width=600)

In [ ]:
amps_along_axis =  np.max(abs(p_rec[:,:,24]),axis=0)
plt.figure()
plt.plot(amps_along_axis)

### Loop across frequencies

In [ ]:
# loop over frequencies
sd_sr = Sim.sr
freqs = np.array([20000, 10000, 5000, 1000, 500, 100])
pulse_durations = np.array([max(7/freq,20*0.16/1500) for freq in freqs])
amps = []
for (freq,dur) in zip(freqs, pulse_durations):
  sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=freq, pulse_duration=dur)
  start = int((dur*sd_sr)//3) #cut beginning
  sd = sd[start:] 
  p_rec, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=15)
  # pressure video
  fn_p =  f"./pressure_gp{freq}Hz.mp4"
  writevid(fn_p, p_rec, len(p_rec)//30)
  amps_along_axis =  np.max(abs(p_rec[:,:,24]),axis=0)
  amps.append(amps_along_axis)

In [ ]:
plt.figure(figsize=(12,12))
for i, freq in enumerate(freqs):
  plt.plot(amps[i], label = f"{freq}Hz")
plt.legend()
plt.savefig("pAttenuation_bottleneck.png")

## Hydrophone array as motion sensor

### Define Mesh

In [ ]:
mesh = create_mesh3(with_sensors=False, with_emitters=False)
mesh['p_emitters'] = (((16, 24, 15))) # single emitter
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

### Measure p and v in plane and obtain the same v again from p

In [ ]:
# motion along x
# loop over frequencies
sd_sr = Sim.sr
dx = Sim.mesh['dx']
freqs = np.array([20000, 10000, 5000, 1000, 500, 100])
pulse_durations = np.array([max(7/freq,20*0.16/1500) for freq in freqs])
amps = []
for (freq,dur) in zip(freqs, pulse_durations):
  sd = 6000 * np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=freq, pulse_duration=dur)
  start = int((dur*sd_sr)//3) #cut beginning
  sd = sd[start:] 
  p, v0, v1 = Sim.rec_pv_plane(sd, sd_sr, plane_axis=2, plane_pos=15)
  
  v0_from_p = np.cumsum(-np.gradient(p,axis=1)/1000/dx, axis=0)/sd_sr
  v1_from_p = np.cumsum(-np.gradient(p,axis=2)/1000/dx, axis=0)/sd_sr

  hdistance = 6 #how many dx between hydrophones?
  v0_from_p_coarse = np.empty(v0.shape)
  v1_from_p_coarse = np.empty(v1.shape)

  for i in range(hdistance):
    v0_from_p_coarse[:,i::hdistance,:] = np.cumsum(-np.gradient(p[:,i::hdistance,:],axis=1)/1000/(hdistance*dx), axis=0)/sd_sr
    v1_from_p_coarse[:,:,i::hdistance] = np.cumsum(-np.gradient(p[:,:,i::hdistance],axis=2)/1000/(hdistance*dx), axis=0)/sd_sr

  # video export
  dx_cm = 100*dx

  writevid(f"./velocity_gp{freq}Hz_v0.mp4", v0, len(v0)//30)
  writevid(f"./velocity_gp{freq}Hz_v0_from_p_dx{dx_cm:.1g}cm.mp4", v0_from_p, len(v0_from_p)//30)
  writevid(f"./velocity_gp{freq}Hz_v0_from_p_dx{hdistance*dx_cm:.1g}cm.mp4", v0_from_p_coarse, len(v0_from_p_coarse)//30)

  writevid(f"./velocity_gp{freq}Hz_v1.mp4", v1, len(v1)//30)
  writevid(f"./velocity_gp{freq}Hz_v1_from_p_dx{dx_cm:.1g}cm.mp4", v1_from_p, len(v1_from_p)//30)
  writevid(f"./velocity_gp{freq}Hz_v1_from_p_dx{hdistance*dx_cm:.1g}cm.mp4", v1_from_p_coarse, len(v1_from_p_coarse)//30)

In [ ]:
plt.figure()
plt.imshow(v0[500].T[::-1])
plt.colorbar()
plt.figure()
plt.imshow(p[500].T[::-1])
plt.figure()
plt.imshow(v0_from_p[500].T[::-1])
plt.colorbar()
plt.figure()
plt.imshow(v0_from_p_coarse[500].T[::-1])
plt.colorbar()

## Hydrophone array as source finder #todo

### Define Mesh

In [ ]:
mesh = create_mesh3(with_sensors=True, with_emitters=False)
mesh['p_emitters'] = (((18, 19, 15))) # single emitter
# only water
mesh['c'] = 1500*np.ones(mesh['c'].shape)
mesh['rho'] = 1000*np.ones(mesh['rho'].shape)

Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

In [ ]:
# distance between sensor and emitter
l0, l1, _ = np.array(mesh['p_emitters']) - np.array(Sim.mesh['p_sensors'][0])
print(Sim.mesh['p_sensors'])
print(mesh['p_emitters'])
print(l0*Sim.mesh['dx'],l1*Sim.mesh['dx']) # should be recovered

### Define Sound

In [ ]:
sd_sr = Sim.sr
dx = Sim.mesh['dx']

dur = 0.005
f = 10000
sd = 6000 * np_ramped_sinusoid(samplerate=sd_sr, freq=f, duration=dur,ramp_time=0.0005)
sd = 6000 * np_gauss
start = int((dur*sd_sr)//3) #cut beginning
sd = sd[start:] 
data = Sim.rec_p_sensors(sd, sd_sr)

In [ ]:
#@title Define observables in SI units
p_sensors = Sim.mesh['p_sensors']
dx = Sim.mesh['dx']

def from_p_sensors_get_pressure(data):
    """
    define what hydrophone measures the relevant pressure
    :param data: recording from all input hydrophones in order of p_sensors
    :return: pressure
    """
    ch = 0
    return data[:,ch]

def from_p_sensors_get_velocity_0(data):
    """
    define which hydrophones measure the relevant acceleration
    :param data: recording from all input hydrophones in order of p_sensors
    :return: acceleration
    """
    ch_pair = [1, 2]
    distance = dx * (p_sensors[ch_pair[1]][0] - p_sensors[ch_pair[0]][0])
    rho0_between =  1000 # todo: automatic

    # incompressible Euler equation
    a = - (data[:, ch_pair[1]] - data[:, ch_pair[0]]) / distance / rho0_between
    v = np.cumsum(a)/sd_sr
    return v

def from_p_sensors_get_velocity_1(data):
    """
    define which hydrophones measure the relevant acceleration
    :param data: recording from all input hydrophones in order of p_sensors
    :return: acceleration
    """
    ch_pair = [3, 4]
    distance = dx * (p_sensors[ch_pair[1]][1] - p_sensors[ch_pair[0]][1])
    rho0_between =  1000 # todo: automatic

    # incompressible Euler equation
    a = - (data[:, ch_pair[1]] - data[:, ch_pair[0]]) / distance / rho0_between
    v = np.cumsum(a)/sd_sr
    return v

### Compute p and v

In [ ]:
# Pressure and velocity in time domain
p = from_p_sensors_get_pressure(data)
v0 = from_p_sensors_get_velocity_0(data)
v1 = from_p_sensors_get_velocity_1(data)
plt.figure()
rhoc = 1500*1000
plt.plot(p, label = "Pressure")
plt.plot(v0*rhoc, label = "Velocity (1/rhoc), ax0")
plt.plot(v1*rhoc, label = "Velocity (1/rhoc), ax1")
plt.legend()

### Find p-v ratio for each Fourier component

Find the amplitude ratio and phase of pressure and velocity at a dominant frequency component for the initial waveform.
As the pv relationships for a monopole (in open water) is known, these can be used to estimate the location of the monopole sound source (in a tank).

In [ ]:
stop = 1200 #todo define initial (echoe-free) waveform automatically

In [ ]:
# pv relationship is known for a monopole for each frequency component
# plot p,v frequency components
pfft = np.fft.rfft(p[:stop])
v0fft = np.fft.rfft(v0[:stop])
v1fft = np.fft.rfft(v1[:stop])
fs = np.fft.rfftfreq(stop) * sd_sr
plt.figure()
plt.plot(fs, pfft.real, label = "Pressure")
plt.plot(fs, rhoc*v0fft.real, label = "Velocity (1/rhoc), ax0")
plt.plot(fs, rhoc*v1fft.real, label = "Velocity (1/rhoc), ax1")
plt.legend()
plt.xlim([0,4*f])
plt.xlabel('Freq [Hz]')

In [ ]:
# only consider dominant frequency modes
v = np.sqrt(v0fft**2+v1fft**2)
v_threshold = np.percentile(abs(v),95)
print(v_threshold*rhoc)
idx = v>v_threshold
idx[0] = 0 # exclude dc component

In [ ]:
# pvratios
plt.figure()
plt.plot(fs[idx], np.abs(pfft[idx]/v0fft[idx]), label = "abs(p/v), ax0")
plt.plot(fs[idx], np.abs(pfft[idx]/v1fft[idx]), label = "abs(p/v), ax1")
plt.xlim([0,4*f])
plt.xlabel('Freq [Hz]')
plt.legend()

In [ ]:
# pvphases
plt.figure()
plt.plot(fs[idx], np.angle(pfft[idx]/v0fft[idx])/np.pi, label = "angle(p/v), ax0")
plt.plot(fs[idx], np.angle(pfft[idx]/v1fft[idx])/np.pi, label = "angle(p/v), ax1")
plt.xlim([0,4*f])
plt.xlabel('Freq [Hz]')
plt.legend()

In [ ]:
def r_from_z(z,k,rhoc):
  """
  z= p/v is complex
  z = A/B * exp(i*dphi)
  """
  r = 1j / (k * (rhoc/z - 1)) # if p and v ~ exp(-iwt)
  return r #should be real for existing solution

In [ ]:
# Compute distance estimates
# 1. based on p/v phase
k = 2*np.pi*fs/1500
r0s = r_from_z(pfft[idx]/v0fft[idx],k[idx],rhoc)
r1s = r_from_z(pfft[idx]/v1fft[idx],k[idx],rhoc)
print(fs[idx])
print(r0s,r1s)
print(abs(r0s),abs(r1s))

## Imuluse-response based sound conditioning

### Option A) via pressure sensors

#### Define Mesh

In [ ]:
mesh = create_mesh3(with_sensors=True, with_emitters=True)
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

#### Record impulse responses for each speaker

In [ ]:
sd_sr = Sim.sr
irs, irs_meta = record_irs_p_sensors(Sim, sd_sr)

In [ ]:
for i, ir in enumerate(irs):
  plot_timeseries(ir, irs_meta['sd_sr'], title = f"Impulse Response #{i}", lim = abs(ir).max())

#### Define pressure-sensor derived observables

In [ ]:
p_sensors = Sim.mesh['p_sensors']
dx = Sim.mesh['dx']

def from_p_sensors_get_pressure(data):
    """
    define what hydrophone measures the relevant pressure
    :param data: recording from all input hydrophones in order of p_sensors
    :return: pressure
    """
    ch = 0
    return data[:,ch]

def from_p_sensors_get_acceleration_x(data):
    """
    define which hydrophones measure the relevant acceleration
    :param data: recording from all input hydrophones in order of p_sensors
    :return: acceleration
    """
    ch_pair = [1, 2]
    distance = dx * (p_sensors[ch_pair[1]][0] - p_sensors[ch_pair[0]][0])
    rho0_between =  1000 # todo: automatic

    # incompressible Euler equation
    a = - (data[:, ch_pair[1]] - data[:, ch_pair[0]]) / distance / rho0_between
    return a

def from_p_sensors_get_acceleration_y(data):
    """
    define which hydrophones measure the relevant acceleration
    :param data: recording from all input hydrophones in order of p_sensors
    :return: acceleration
    """
    ch_pair = [3, 4]
    distance = dx * (p_sensors[ch_pair[1]][1] - p_sensors[ch_pair[0]][1])
    rho0_between =  1000 # todo: automatic

    # incompressible Euler equation
    a = - (data[:, ch_pair[1]] - data[:, ch_pair[0]]) / distance / rho0_between
    return a

#### Make Kernels from IRs

In [ ]:
obs = {"pressure": from_p_sensors_get_pressure,
       "acceleration_x": from_p_sensors_get_acceleration_x}
kns = make_kernels(irs, irs_meta, obs)

In [ ]:
plot_kernels(kns,irs_meta['sd_sr'])

#### Define Target Waveform

In [ ]:
# target waveform
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
wav = np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=800, pulse_duration=0.01)
plot_timeseries(wav,sd_sr)

In [ ]:
# define target observables (SI units)
target_list = [{"pressure": 100 * wav, "acceleration_x": 0 * wav},
               {"pressure":   0 * wav, "acceleration_x": 5 * wav}]
target_handles = ["PressureOnly", "MotionOnly"]

#### Calculate conditioned sounds

In [ ]:
# which speakers should be active?
# (n speakers = n observables)
p_emitters = Sim.mesh['p_emitters']
sp = np.identity(len(p_emitters), dtype=bool)[:len(obs)].tolist()
print(sp) # sound dimension to speaker mapping

# calculate conditioned sounds
sounds = {}
for handle, tg in zip(target_handles, target_list):
  sound = find_sound(ks=kns, tg=tg, sp=sp)
  sounds[handle] = sound

In [ ]:
# look at generated p_emitter loadings that can produce the targets
h = 'MotionOnly'
sd = sounds[h]
t = np.arange(sd.shape[0])/sd_sr
plt.plot(t*1000, sd) # a curve for each emitter

In [ ]:
# look at generated p_emitter loadings that can produce the targets
h = 'PressureOnly'
sd = sounds[h]
t = np.arange(sd.shape[0])/sd_sr
plt.plot(t*1000, sd) # a curve for each emitter

#### Run Simulation: with conditioned sounds 

In [ ]:
sds_rec = {}
for handle in target_handles:
  print(handle)
  sds_rec[handle] = Sim.rec_p_sensors(sounds[handle], sd_sr)

In [ ]:
# plot
for i, handle in enumerate(target_handles):
  print(handle)
  plot_sound_simulation(handle, sds_rec[handle], sounds[handle], target_list[i], kns, sd_sr, obs)

#### Extra: how localized is the conditioning?

In [ ]:
# Shift sensors along x
Sim.mesh['p_sensors']

In [ ]:
shiftby = 4
Sim.mesh['p_sensors'] = tuple((x-shiftby,y,z) for (x,y,z) in Sim.mesh['p_sensors'])
Sim.mesh['p_sensors']

In [ ]:
plot_mesh(**Sim.mesh)

In [ ]:
sds_rec = {}
for handle in target_handles:
  print(handle)
  sds_rec[handle] = Sim.rec_p_sensors(sounds[handle], sd_sr)

In [ ]:
# plot
for i, handle in enumerate(target_handles):
  print(handle)
  plot_sound_simulation(handle, sds_rec[handle], sounds[handle], target_list[i], kns, sd_sr, obs)

### Option B) via plane recordings  #todo
todo this section is at its early experimental stage...

#### Define Mesh

In [ ]:
mesh = create_mesh3(with_emitters=True)
Sim = WeeFDTD(mesh)
plot_mesh(**Sim.mesh)

In [ ]:
Sim.mesh['p_emitters']

#### Record impulse responses in a plane

In [ ]:
sd_sr = Sim.sr
plane_axis = 2
plane_pos = Sim.mesh['volume_shape'][plane_axis]//2
irs_p, irs_v0, irs_v1, irs_meta = record_irs_pv_plane(Sim, sd_sr, plane_axis=plane_axis,plane_pos=plane_pos)

In [ ]:
# pressure video
ir_p = irs_p[3]
fn_p =  "./pressure.mp4"
writevid(fn_p, ir_p, len(ir_p)//50)
showvid(fn_p,width=300)

In [ ]:
irs_p[0].shape

#### Define derived observables

A) Target area for pressure and motion at center

In [ ]:
dx = Sim.mesh['dx']

# optimize for mean pressure or acceleration within target area
# target area must be small enough for average to make sense todo: find more reasonable area-wide pooling

vol_shape = Sim.mesh['volume_shape']
plane_shape = np.delete(vol_shape,plane_axis)
c0, c1 = plane_shape//2
pm = 2 #extend in grid points
img = np.mean(Sim.mesh['c'], axis=plane_axis)


img[c0-pm:c0+pm,c1-pm:c1+pm] = 0
plt.imshow(img.T[::-1])


def from_p_plane_get_pressure(pplane):
    """
    on pressure plane
    """
    mean_p = np.mean(pplane[:,c0-pm:c0+pm,c1-pm:c1+pm],axis=(1,2))
    return mean_p

def from_p_plane_get_acceleration_0(pplane):
    """
    on pressure plane
    """
    rho0_between =  1000 # todo: automatic
    # incompressible Euler equation
    mean_a = - np.mean(np.gradient(pplane[:,c0-pm:c0+pm,c1-pm:c1+pm],axis=1), axis=(1,2)) / dx / rho0_between
    return mean_a

def from_p_plane_get_acceleration_1(pplane):
    """
    on pressure plane
    """
    rho0_between =  1000 # todo: automatic
    # incompressible Euler equation
    mean_a = - np.mean(np.gradient(pplane[:,c0-pm:c0+pm,c1-pm:c1+pm],axis=2), axis=(1,2)) / dx / rho0_between
    return mean_a

B) Several target areas for pressure

In [ ]:
# Target areas for pressure
dx = Sim.mesh['dx']

# optimize for mean pressure or acceleration within target area
# target area must be small enough for average to make sense

vol_shape = Sim.mesh['volume_shape']
plane_shape = np.delete(vol_shape,plane_axis)
c0, c1 = plane_shape//2
pm = 3 #even extend in grid points
img = np.mean(Sim.mesh['c'], axis=plane_axis)

img[c0-2*pm:c0-0*pm,c1-1*pm:c1-0*pm] = 0
img[c0-2*pm:c0-0*pm,c1+0*pm:c1+1*pm] = 300
img[c0+0*pm:c0+2*pm,c1-1*pm:c1-0*pm] = 600
img[c0+0*pm:c0+2*pm,c1+0*pm:c1+1*pm] = 900
plt.imshow(img.T[::-1])

def parea1(pplane):
  return np.mean(pplane[:,c0-2*pm:c0-0*pm,c1-1*pm:c1-0*pm],axis=(1,2))
def parea2(pplane):
  return np.mean(pplane[:,c0-2*pm:c0-0*pm,c1+0*pm:c1+1*pm],axis=(1,2))
def parea3(pplane):
  return np.sum(abs(pplane[:,c0+0*pm:c0+1*pm,c1-1*pm:c1-0*pm]),axis=(1,2))
def parea4(pplane):
  return np.sum(abs(pplane[:,c0+0*pm:c0+1*pm,c1+0*pm:c1+1*pm]),axis=(1,2))

#### Make Kernels from IRs

In [ ]:
obs = {"pressure": from_p_plane_get_pressure,
       "acceleration_x": from_p_plane_get_acceleration_1}

obs = {"parea1": parea1,
       "parea2": parea2,
       "parea3": parea3,
       "parea4": parea4}

kns = make_kernels(irs_p, irs_meta, obs)

In [ ]:
plot_kernels(kns,irs_meta['sd_sr'])

#### Define Target Waveform

In [ ]:
# target waveform
sd_sr = Sim.sr # Sound samplerate. Simulation resamples the sound if there is no match.
wav = np_gaussian_double_pulse(samplerate=sd_sr, center_frequency=400, pulse_duration=0.01)
plot_timeseries(wav,sd_sr)

In [ ]:
# define target observables (SI units)
target_list = [{"pressure": 100 * wav, "acceleration_x": 0 * wav},
               {"pressure":   0 * wav, "acceleration_x": 5 * wav}]
target_handles = ["PressureOnly", "MotionOnly"]


target_list = [{"parea1": 100 * wav, "parea2": 100 * wav, "parea3": 0*wav, "parea4": 0*wav}]
target_handles = ["Left"]

#### Calculate conditioned sounds

In [ ]:
# which speakers should be active?
# (n speakers = n observables)
p_emitters = Sim.mesh['p_emitters']
sp = np.identity(len(p_emitters), dtype=bool)[:len(obs)].tolist()
print(sp) # sound dimension to speaker mapping

# calculate conditioned sounds
sounds = {}
for handle, tg in zip(target_handles, target_list):
  sound = find_sound(ks=kns, tg=tg, sp=sp)
  sounds[handle] = sound

In [ ]:
# look at generated p_emitter loadings that can produce the targets
h = 'MotionOnly'
sd = sounds[h]
t = np.arange(sd.shape[0])/sd_sr
plt.plot(t*1000, sd) # a curve for each emitter

In [ ]:
# look at generated p_emitter loadings that can produce the targets
h = 'PressureOnly'
h = 'Left'
sd = sounds[h]
t = np.arange(sd.shape[0])/sd_sr
plt.plot(t*1000, sd) # a curve for each emitter

#### Run Simulation: with conditioned sounds 

In [ ]:
sds_rec = {}
for handle in target_handles:
  print(handle)
  sds_rec[handle], _, _ = Sim.rec_pv_plane(sounds[handle], sd_sr, plane_axis=plane_axis, plane_pos=plane_pos)

In [ ]:
# pressure video
handle = "Area1"
handle = 'Left'
rec = sds_rec[handle]
fn_p =  "./pressure.mp4"
writevid(fn_p, rec, len(rec)//30)
showvid(fn_p,width=600)

In [ ]:
# plot
for i, handle in enumerate(target_handles):
  print(handle)
  plot_sound_simulation(handle, sds_rec[handle], sounds[handle], target_list[i], kns, sd_sr, obs)